## UPDATED SOLO PROJECT BUT JUST ANALYSIS

In [ ]:
import numpy as np
import sunpy
import astropy
import numpy
from IPython.core.pylabtools import figsize
from asdf.commands.diff import print_in_tree
from matplotlib import pyplot as plt
# import cdflib
# import sunpy_soar
from matplotlib.pyplot import legend
from sunpy.net import Fido, attrs as a
from sunpy.sun.constants import density
import pandas as pd

# Set display options to show all columns
pd.set_option('display.max_columns', None)


## DATA MANIPULATION

In [ ]:
SWA_pickle_files = ['./Date_Ranges_SOLO_YR1/SOLO_SWA_date_range1-25.pkl','./Date_Ranges_SOLO_YR1/SOLO_SWA_date_range26-51.pkl','./Date_Ranges_SOLO_YR1/SOLO_SWA_date_range52-77.pkl','./Date_Ranges_SOLO_YR1/SOLO_SWA_date_range78-103.pkl','./Date_Ranges_SOLO_YR1/SOLO_SWA_date_range104-129.pkl','./Date_Ranges_SOLO_YR1/SOLO_SWA_date_range130-134.pkl']
MAG_pickle_files = ['./Date_Ranges_SOLO_YR1/SOLO_MAG_date_range1-25.pkl','./Date_Ranges_SOLO_YR1/SOLO_MAG_date_range26-51.pkl','./Date_Ranges_SOLO_YR1/SOLO_MAG_date_range52-77.pkl','./Date_Ranges_SOLO_YR1/SOLO_MAG_date_range78-103.pkl','./Date_Ranges_SOLO_YR1/SOLO_MAG_date_range104-129.pkl','./Date_Ranges_SOLO_YR1/SOLO_MAG_date_range130-134.pkl']


In [ ]:
# Convert the timeseries to a dataframe
import pickle

SWA_timeseries_df = None
# Unpickling (loading) an object
all_SWA_dfs = []
for path in SWA_pickle_files:
            # Use pandas to read the pickled file
    a_SWA_df = pd.read_pickle(path)
    all_SWA_dfs.append(a_SWA_df)
SWA_timeseries_df = pd.concat(all_SWA_dfs,sort=True)

MAG_timeseries_df = None
all_MAG_dfs = []
for path in MAG_pickle_files:
            # Use pandas to read the pickled file
    a_MAG_df = pd.read_pickle(path)
    all_MAG_dfs.append(a_MAG_df)
MAG_timeseries_df = pd.concat(all_MAG_dfs,sort=True)



display(SWA_timeseries_df.head())
MAG_timeseries_df.head()

In [ ]:
print('SWA:', SWA_timeseries_df.columns)
print('MAG:', MAG_timeseries_df.columns)

### Some small statistics for each date range

In [ ]:
SWA_timeseries_df.reset_index()

In [ ]:
SWA_index = SWA_timeseries_df.sort_index(ascending=True).reset_index()
print('SWA Epoch range:', SWA_index['Epoch'].min(), ' to ', SWA_index['Epoch'].max())
# Calculate seconds diff between current measurement before the previous one
SWA_index["Seconds_diff"]= SWA_index.groupby('name')['Epoch'].diff().dt.total_seconds()
# Calculate statistics
SWA_stats = SWA_index.groupby("name").agg( minDateTime=('Epoch', 'min'), maxDateTime=('Epoch', 'max'), 
                                          avgMeasurementFrequencySeconds=('Seconds_diff', 'mean') ,stdMeasurementFrequencySeconds=('Seconds_diff', 'std') ,
                                          countMeasurements=('Epoch', 'count') , totalDurationHours = ( 'Epoch' , lambda x: (x.max() - x.min()).total_seconds() / 3600 )  , totalDurationDays = ( 'Epoch' , lambda x: (x.max() - x.min()).total_seconds() / 86400 )  ,  )
SWA_stats

In [ ]:
MAG_index = MAG_timeseries_df.sort_index(ascending=True).reset_index()
print('MAG Epoch range:', MAG_index['EPOCH'].min(), ' to ', MAG_index['EPOCH'].max())
# Calculate seconds diff between current measurement before the previous one
MAG_index["Seconds_diff"]= MAG_index.groupby('name')['EPOCH'].diff().dt.total_seconds()
# Calculate statistics
MAG_stats = MAG_index.groupby("name").agg( minDateTime=('EPOCH', 'min'), maxDateTime=('EPOCH', 'max'), 
                                          avgMeasurementFrequencySeconds=('Seconds_diff', 'mean') ,stdMeasurementFrequencySeconds=('Seconds_diff', 'std') ,
                                          countMeasurements=('EPOCH', 'count') , totalDurationHours = ( 'EPOCH' , lambda x: (x.max() - x.min()).total_seconds() / 3600 )  , totalDurationDays = ( 'EPOCH' , lambda x: (x.max() - x.min()).total_seconds() / 86400 )  ,  )
MAG_stats

# MAIN CODE

In [ ]:
# # Check if the column 'TxTyTz_RTN_0' exists in the DataFrame
# if 'TxTyTz_RTN_0' in SWA_timeseries_df.columns:
#     print("Column 'TxTyTz_RTN_0' exists in the DataFrame.")
# else:
#     print("Column 'TxTyTz_RTN_0' does not exist in the DataFrame.")
#
# # Display the first few rows of the DataFrame
# display(SWA_timeseries_df)


In [ ]:
#rename columns to more manageable names
renameDict = {'V_RTN_0':'VR', 'V_RTN_1':'VT', 'V_RTN_2':'VN', # VELOCITY COLUMNS 
              'B_RTN_0':'BR', 'B_RTN_1':'BT', 'B_RTN_2':'BN', # MAGNETIC FIELD COLUMNS
              'TxTyTz_RTN_0':'T_R', 'TxTyTz_RTN_1':'T_T', 'TxTyTz_RTN_2':'T_N', # TEMPERATURE COLUMNS
              'P_RTN_0':'P_RR','P_RTN_1':'P_TT','P_RTN_2':'P_NN','P_RTN_3':'P_RT','P_RTN_4':'P_RN','P_RTN_5':'P_TN' # PRESSURE TENSOR COLUMNS
    }
MAG_timeseries_df.rename(columns=renameDict, inplace=True)
SWA_timeseries_df.rename(columns=renameDict, inplace=True)

constant_columns = ["Distance", "name"]

#filter the columns we want to work with and make copies of the dataframes
SWA_df = SWA_timeseries_df[['VR', 'VT', 'VN', 'N','T_R','T_T','T_N','T','P_RR','P_TT','P_NN','P_RT','P_RN','P_TN'] + constant_columns].copy()
MAG_df = MAG_timeseries_df[['BR', 'BT', 'BN'] + constant_columns].copy()

#display just the first few 5 rows of each dataframe-- tail for the last 5 rows
display(SWA_df.head())
display(MAG_df.head())


In [ ]:
TEST_T=((SWA_df['T_R'][0]+SWA_df['T_T'][0]+SWA_df['T_N'][0])/3) # average T
TEST_T

In [ ]:
def eV_to_K(T_eV):
    return T_eV * 11604.518

SWA_df['T'] = SWA_df['T'].apply(eV_to_K)
SWA_df['T_R'] = SWA_df['T_R'].apply(eV_to_K)
SWA_df['T_T'] = SWA_df['T_T'].apply(eV_to_K)
SWA_df['T_N'] = SWA_df['T_N'].apply(eV_to_K)

SWA_df.head()


In [ ]:
def P_to_T(P,N):
    return P/(N*1.38*10**-23)

#remember P is in J/cm^3, N is in cm^-3, kB=1.38e-23 J/K so T = P/(N*kB) is in K

SWA_df['T_RR'] = P_to_T(SWA_df['P_RR'],SWA_df['N'])
SWA_df['T_RT'] = P_to_T(SWA_df['P_RT'],SWA_df['N'])
SWA_df['T_RN'] = P_to_T(SWA_df['P_RN'],SWA_df['N'])
SWA_df['T_TT'] = P_to_T(SWA_df['P_TT'],SWA_df['N'])
SWA_df['T_TN'] = P_to_T(SWA_df['P_TN'],SWA_df['N'])
SWA_df['T_NN'] = P_to_T(SWA_df['P_NN'],SWA_df['N'])

SWA_df.head()

In [ ]:
# List of original pressure tensor columns
pressure_cols = ['P_RR', 'P_RT', 'P_RN', 'P_TT', 'P_TN', 'P_NN']

# Convert J/cm^3 → nPa (multiply by 1e15) as 1J/cm^3 = 10^6 J/m^3 = 10^6 Pa = 10^15 nPa
for col in pressure_cols:
    SWA_df[col + '_nPa'] = SWA_df[col] * 1e15

SWA_df.head()


In [ ]:
import numpy as np
MAG_df['|B|'] = np.linalg.norm(MAG_df[['BR', 'BT', 'BN']], axis=1)
SWA_df['|V|'] = np.linalg.norm(SWA_df[['VR', 'VT', 'VN']], axis=1)


MAG_df.head()

In [ ]:
SWA_df.head()

In [ ]:
#import plotly for interactive plots
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go

In [ ]:
# # Loop through each date range in the daterange dictionary
# for key in all_Date_Range_info:
#     start_date = all_Date_Range_info[key]['time_range'].start.value  # Get the start date
#     end_date = all_Date_Range_info[key]['time_range'].end.value      # Get the end date
#     name = all_Date_Range_info[key]['name']                          # Get the name of the range
# 
#     # Filter the dataframes for the current date range
#     SWA_filtered = SWA_df[start_date:end_date]
#     MAG_filtered = MAG_df[start_date:end_date]
# 
#     # Create subplots for plotting                       1st number is the width, 2nd is the height
#     fig, ax = plt.subplots(3, 1, layout='tight', sharex=True, figsize=(8, 8))
# 
#     # Plotting for Proton Velocity Moments
#     ax[0].set_title(f'Radial Velocity: {name}')
#     ax[0].plot(SWA_filtered.VR, color='red', label='VR')# ,linewidth=0.2)
#     ax[0].tick_params(top=False, bottom=True, left=True, right=False, labelbottom=True, labeltop=False, labelleft=True, labelright=False)
#     ax[0].legend()
# 
#     # Plotting for Magnetic Fields
#     ax[1].set_title(f'Magnetic Fields: {name}')
#     ax[1].plot(MAG_filtered.BT, color='green', label='BT', linewidth=0.2)
#     ax[1].plot(MAG_filtered.BN, color='orange', label='BN', linewidth=0.2)
#     ax[1].plot(MAG_filtered['|B|'], color='black', label='|B|', linewidth=0.2)
#     ax[1].plot(MAG_filtered.BR, color='red', label='BR', linewidth=0.2)
#     ax[1].tick_params(top=False, bottom=True, left=True, right=False, labelbottom=True, labeltop=False, labelleft=True, labelright=False)
#     ax[1].legend()
# 
#     # Plotting for Density
#     ax[2].set_title(f'Density: {name}')
#     ax[2].plot(SWA_filtered.N, color='blue', label='N')
#     ax[2].tick_params(top=False, bottom=True, left=True, right=False, labelbottom=True, labeltop=False, labelleft=True, labelright=False)
#     ax[2].legend()
# 
#     fig.tight_layout( )
#     plt.subplots_adjust(hspace=0.3)  # Increase the space between the subplots
#     # Format the dates nicely
#     fig.autofmt_xdate()
# 
#     for an_ax in ax:
#         an_ax.get_legend().remove()
#     # Show each figure
#     # fig.show()
#     plotly_fig = tls.mpl_to_plotly(fig)
#     legend = go.layout.Legend(
#         x=1,
#         y=0.5
#     )
#     plotly_fig.update_layout(showlegend=True,legend=legend)
#     plotly_fig.update_xaxes(matches='x')
#     iplot(plotly_fig)
#     #break

# TASK 2.2

In [ ]:
SWA_df.index

In [ ]:
# We want to create a new dataframe that will contain the specific data from the different date ranges
# Loop through each date range in the daterange dictionary
import pandas as pd


# OLD WAY TO DO IT - NOW THIS IS DONE IN SAVE_FILES SCRIPT!!
# SWA_df = pd.DataFrame()
# MAG_df = pd.DataFrame()
# for key in all_Date_Range_info:
#     start_date = all_Date_Range_info[key]['time_range'].start.value  # Get the start date
#     end_date = all_Date_Range_info[key]['time_range'].end.value      # Get the end date
#     name = all_Date_Range_info[key]['name']                          # Get the name of the range
#     distance = all_Date_Range_info[key]['distance']                  # Get the distance of the range

#     # Filter the dataframes for the current date range
#     SWA_filtered = SWA_df.sort_index()[start_date:end_date].copy()
#     SWA_filtered['Distance'] = distance
#     SWA_filtered['name'] = name

#     MAG_filtered = MAG_df.sort_index()[start_date:end_date].copy()
#     MAG_filtered['Distance'] = distance
#     MAG_filtered['name'] = name

#     SWA_df = pd.concat([SWA_df, SWA_filtered])
#     MAG_df = pd.concat([MAG_df, MAG_filtered])




In [ ]:
SWA_df = SWA_df.reset_index().drop_duplicates().set_index('Epoch').sort_index()
MAG_df = MAG_df.reset_index().drop_duplicates().set_index('EPOCH').sort_index()

In [ ]:
print(SWA_df.name.value_counts())
MAG_df.name.value_counts()

In [ ]:
MAG_df['|B_transv|'] = np.linalg.norm(MAG_df[['BT', 'BN']], axis=1)
MAG_df.head()

In [ ]:
AU_M   = 1.495978707e11     # m
KM_TO_M = 1e3               # m to m
CM3_TO_M3 = 1e6             # cm^-3 to m^-3
m_p = 1.6726219e-27         # kg


SWA_df['mass_flux'] = SWA_df['N']* CM3_TO_M3 * SWA_df['|V|']*KM_TO_M*((SWA_df['Distance']*AU_M)**2)*m_p
print(SWA_df)


In [ ]:
# Calculate mean values for SWA and MAG data

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

SWA_df_mean = SWA_df.groupby('name')[['VR','VN','VT','|V|','N','Distance','mass_flux','T_R','T_T','T_N','T','P_RR','P_RT','P_RN','P_TT','P_TN','P_NN','P_RR_nPa','P_RT_nPa','P_RN_nPa','P_TT_nPa','P_TN_nPa','P_NN_nPa','T_RR','T_RT','T_RN','T_TT','T_TN','T_NN']].mean()
MAG_df_mean = MAG_df.groupby('name')[['BR','BT','BN','|B|','|B_transv|']].mean()

print(SWA_df_mean.reset_index().name.value_counts())
MAG_df_mean.reset_index().name.value_counts()

In [ ]:
# Add absolute values for magnetic field components
MAG_df_mean['abs_BR'] = np.abs(MAG_df_mean['BR'])
MAG_df_mean['abs_BT'] = np.abs(MAG_df_mean['BT'])
MAG_df_mean['abs_BN'] = np.abs(MAG_df_mean['BN'])

# Merge SWA and MAG mean data
mean_values_df = pd.merge(SWA_df_mean, MAG_df_mean, on='name').rename(columns=lambda x: x + '_mean')
mean_values_df

In [ ]:
print(mean_values_df.columns.tolist())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

# ----------------------------------------------------
# Angles + |B|
# ----------------------------------------------------
mean_values_df['theta'] = np.degrees(
    np.arctan(mean_values_df['BT_mean'] / mean_values_df['BR_mean'])
)

mean_values_df['phi'] = np.degrees(
    np.arctan(mean_values_df['|B_transv|_mean'] / mean_values_df['BR_mean'])
)

mean_values_df['|mean_B_vector|'] = np.linalg.norm(
    mean_values_df[['BR_mean', 'BT_mean', 'BN_mean']], axis=1
)

# ----------------------------------------------------
# Units
# ----------------------------------------------------
UNITS = {
    'VR_mean': 'km/s', 'VN_mean': 'km/s', 'VT_mean': 'km/s', '|V|_mean': 'km/s',
    'N_mean': r'cm$^{-3}$',
    'Distance_mean': 'AU',
    'mass_flux_mean': 'kg/s',

    'T_mean': 'K','T_R_mean': 'K', 'T_T_mean': 'K', 'T_N_mean': 'K',
    'T_RR_mean': 'K', 'T_RT_mean': 'K', 'T_RN_mean': 'K',
    'T_TT_mean': 'K', 'T_TN_mean': 'K', 'T_NN_mean': 'K',

    'P_RR_mean': 'J/cm$^{3}$',
    'P_RT_mean': 'J/cm$^{3}$',
    'P_RN_mean': 'J/cm$^{3}$',
    'P_TT_mean': 'J/cm$^{3}$',
    'P_TN_mean': 'J/cm$^{3}$',
    'P_NN_mean': 'J/cm$^{3}$',

    'P_RR_nPa_mean': 'nPa',
    'P_RT_nPa_mean': 'nPa',
    'P_RN_nPa_mean': 'nPa',
    'P_TT_nPa_mean': 'nPa',
    'P_TN_nPa_mean': 'nPa',
    'P_NN_nPa_mean': 'nPa',

    'BR_mean': 'nT', 'BT_mean': 'nT', 'BN_mean': 'nT',
    '|B|_mean': 'nT', '|B_transv|_mean': 'nT',
    'abs_BR_mean': 'nT', 'abs_BT_mean': 'nT', 'abs_BN_mean': 'nT',
    '|mean_B_vector|': 'nT',

    'theta': 'deg',
    'phi': 'deg',
}

# ----------------------------------------------------
# Main plotting function
# ----------------------------------------------------
def redo_data_plots(df):

    skip_log_plots = ['VN_mean', 'VT_mean', 'BR_mean',
                      'BT_mean', 'BN_mean', 'theta', 'phi']

    columns = [c for c in df.columns if c != 'Distance_mean']

    for col in columns:

        x = df['Distance_mean'].values
        y = df[col].values

        # =========================================================
        # 1) LINEAR PLOT
        # =========================================================
        mask_lin = np.isfinite(x) & np.isfinite(y)
        if mask_lin.sum() < 2:
            print(f"Skipping linear plot for {col} due to insufficient data.")
            continue

        x_lin = x[mask_lin]
        y_lin = y[mask_lin]

        slope_lin, intercept_lin, _, _, _ = linregress(x_lin, y_lin)

        x_fit_lin = np.linspace(x_lin.min(), x_lin.max(), 200)
        y_fit_lin = slope_lin * x_fit_lin + intercept_lin

        plt.figure()
        sns.scatterplot(x=x_lin, y=y_lin, label=col)
        plt.plot(x_fit_lin, y_fit_lin, 'r--',
                 label=f'Linear Fit (slope = {slope_lin:.2f})')

        plt.title(f'{col} vs Distance')
        plt.xlabel(f"Distance [{UNITS['Distance_mean']}]")
        plt.ylabel(f"{col} [{UNITS.get(col, '')}]")
        plt.grid(True, which='both', alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()

        # =========================================================
        # 2) LOG–LOG PLOT
        # =========================================================
        if col in skip_log_plots:
            continue


        mask_log = (x > 0) & (y > 0) & np.isfinite(x) & np.isfinite(y)
        if mask_log.sum() < 2:
            continue

        x_log = x[mask_log]
        y_log = y[mask_log]

        logx = np.log10(x_log)
        logy = np.log10(y_log)

        slope_pow, intercept_pow, _, _, _ = linregress(logx, logy)

        x_fit_log = np.logspace(np.log10(x_log.min()),
                                np.log10(x_log.max()), 200)
        y_fit_log = 10**intercept_pow * x_fit_log**slope_pow

        plt.figure()
        sns.scatterplot(x=x_log, y=y_log, label=col)
        plt.plot(x_fit_log, y_fit_log, 'r--',
                 label=f'Power-law Fit (slope = {slope_pow:.2f})')

        plt.xscale('log')
        plt.yscale('log')

        # FORCE PRETTY Y-AXIS RANGE (≥ 3 ROUND LOG NUMBERS)
        ymin = y_log.min()
        ymax = y_log.max()

        log_ymin = np.floor(np.log10(ymin))
        log_ymax = np.ceil(np.log10(ymax))

        # Expand if less than 2 digits
        if (log_ymax - log_ymin) < 1:
            log_ymid = 0.5 * (log_ymin + log_ymax)
            log_ymin = np.floor(log_ymid - 1)
            log_ymax = np.ceil(log_ymid + 1)

        plt.ylim(10**log_ymin, 10**log_ymax)

        plt.title(f'{col} vs Distance (log–log scale)')
        plt.xlabel(f"Distance [{UNITS['Distance_mean']}]")
        plt.ylabel(f"{col} [{UNITS.get(col, '')}]")
        plt.grid(True, which='both', alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()

# ----------------------------------------------------
# Run it
# ----------------------------------------------------
redo_data_plots(mean_values_df)


In [ ]:
mean_values_df

In [ ]:
#merge sw_df_extra and MAG_df with mean_values_df respectively as we need all together to calculate deltas
SWA_df_wmean = pd.merge(SWA_df.reset_index(), mean_values_df[['|V|_mean','VR_mean','VN_mean','VT_mean','N_mean','T_R_mean','T_T_mean','T_N_mean','T_mean','T_RR_mean','T_RT_mean','T_RN_mean','T_TT_mean','T_TN_mean','T_NN_mean']].reset_index(), on='name', how='left')

MAG_df_wmean = pd.merge(MAG_df.reset_index(), mean_values_df[['|B|_mean', 'BR_mean', 'BT_mean', 'BN_mean','|B_transv|_mean','|mean_B_vector|','Distance_mean']].reset_index(), on='name', how='left')
SWA_df_wmean

In [ ]:
MAG_df_wmean

In [ ]:
MAG_df_wmean['B0_vector'] = MAG_df_wmean.apply(lambda row: np.array([row['BR_mean'], row['BT_mean'], row['BN_mean']]),axis=1) #Vector of B_0

MAG_df_wmean['B_0_unit_vector']=MAG_df_wmean['B0_vector']/MAG_df_wmean['|mean_B_vector|'] #remember mean_B_vector is B0_vector


MAG_df_wmean['B_vector'] = MAG_df_wmean.apply(lambda row: np.array([row['BR'], row['BT'], row['BN']]),axis=1) #B_total vector

MAG_df_wmean['B_//_0'] = MAG_df_wmean.apply(lambda row: np.dot(row['B_vector'], row['B_0_unit_vector']),axis=1) #scalar B_//_0

MAG_df_wmean['dot_product_check'] = MAG_df_wmean.apply(
    lambda row: (row['B_vector'][0] * row['B_0_unit_vector'][0] +row['B_vector'][1] * row['B_0_unit_vector'][1] +row['B_vector'][2] * row['B_0_unit_vector'][2]),axis=1)

MAG_df_wmean['B_//_0_vector'] = MAG_df_wmean.apply(lambda row: row['B_//_0'] * row['B_0_unit_vector'],axis=1) #vector B_//_0

MAG_df_wmean['B_perp_0_vector'] = (MAG_df_wmean['B_vector'] - MAG_df_wmean['B_//_0_vector']) #vector B_perp_0_


MAG_df_wmean['|B_perp_0|'] = MAG_df_wmean['B_perp_0_vector'].apply(np.linalg.norm) #magnitude of B_perp_0



MAG_df_wmean['B_parallel_0_mean'] = (MAG_df_wmean.groupby('name')['B_//_0'].transform('mean')) #scalar mean B_//_0
MAG_df_wmean['|B_perp_0|_mean'] = (MAG_df_wmean.groupby('name')['|B_perp_0|'].transform('mean')) #mean magnitude of B_perp_0



# Angle between the background magnetic field B0 and the radial direction,
# computed using |BR| so the angle is independent of field polarity
# independent of whether the field points outward or inward


MAG_df_wmean['cos_theta_PS'] = (MAG_df_wmean['BR_mean'].abs() / MAG_df_wmean['|mean_B_vector|'])

# numerical safety
#cos_theta = cos_theta_axis.clip(-1, 1)

MAG_df_wmean['theta_PS_observed'] = np.degrees(np.arccos(MAG_df_wmean['cos_theta_PS']))



# Angle between the background magnetic field B0 and the tangential direction,
# computed using |BT_mean| so the angle is independent of polarity
MAG_df_wmean['phi_PS'] = 90 - MAG_df_wmean['theta_PS_observed']


MAG_df_wmean['theta_PS_theory']= np.degrees(np.arctan(MAG_df_wmean['Distance_mean']*AU_M*2.865e-6/ (400*KM_TO_M))) #Parker spiral angle theory
print(AU_M)
MAG_df_wmean

# Parker Spiral Angles

In [ ]:
#  Radial Scalling of B_PARALLEL_0_mean AND |B_PERP_0|_mean and Parker Spiral Angles

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress


UNITS = {'Distance_mean': 'AU','B_parallel_0_mean': 'nT', '|B_perp_0|_mean': 'nT','theta_PS_observed': 'deg','phi_PS': 'deg'}

# ----------------------------------------------------
# Main plotting function
# ----------------------------------------------------
def redo_data_plots(df):

        columns = ['B_parallel_0_mean','|B_perp_0|_mean','theta_PS_observed','phi_PS']
        skip_log_plots = ['theta_PS_observed', 'phi_PS']

        for col in columns:

            x = df['Distance_mean'].values
            y = df[col].values

            # =========================================================
            # 1) LINEAR PLOT
            # =========================================================
            mask_lin = np.isfinite(x) & np.isfinite(y)
            if mask_lin.sum() < 2:
                continue

            x_lin = x[mask_lin]
            y_lin = y[mask_lin]

            slope_lin, intercept_lin, _, _, _ = linregress(x_lin, y_lin)

            x_fit_lin = np.linspace(x_lin.min(), x_lin.max(), 200)
            y_fit_lin = slope_lin * x_fit_lin + intercept_lin

            plt.figure()
            sns.scatterplot(x=x_lin, y=y_lin, label=col)
            plt.plot(x_fit_lin, y_fit_lin, 'r--',
                     label=f'Linear Fit (slope = {slope_lin:.2f})')

            plt.title(f'{col} vs Distance')
            plt.xlabel(f"Distance [{UNITS['Distance_mean']}]")
            plt.ylabel(f"{col} [{UNITS.get(col, '')}]")
            plt.grid(True, which='both', alpha=0.3)
            plt.legend()
            plt.tight_layout()
            plt.show()

            # =========================================================
            # 2) LOG–LOG PLOT
            # =========================================================
            if col in skip_log_plots:
                continue

            mask_log = (x > 0) & (y > 0) & np.isfinite(x) & np.isfinite(y)
            if mask_log.sum() < 2:
                continue

            x_log = x[mask_log]
            y_log = y[mask_log]

            logx = np.log10(x_log)
            logy = np.log10(y_log)

            slope_pow, intercept_pow, _, _, _ = linregress(logx, logy)

            x_fit_log = np.logspace(np.log10(x_log.min()),
                                    np.log10(x_log.max()), 200)
            y_fit_log = 10**intercept_pow * x_fit_log**slope_pow

            plt.figure()
            sns.scatterplot(x=x_log, y=y_log, label=col)
            plt.plot(x_fit_log, y_fit_log, 'r--',
                     label=f'Power-law Fit (slope = {slope_pow:.2f})')

            plt.xscale('log')
            plt.yscale('log')

            # FORCE PRETTY Y-AXIS RANGE (≥ 3 ROUND LOG NUMBERS)
            ymin = y_log.min()
            ymax = y_log.max()

            log_ymin = np.floor(np.log10(ymin))
            log_ymax = np.ceil(np.log10(ymax))

            # Expand if less than 2 decades
            if (log_ymax - log_ymin) < 1:
                log_ymid = 0.5 * (log_ymin + log_ymax)
                log_ymin = np.floor(log_ymid - 1)
                log_ymax = np.ceil(log_ymid + 1)

            plt.ylim(10**log_ymin, 10**log_ymax)

            plt.title(f'{col} vs Distance (log–log scale)')
            plt.xlabel(f"Distance [{UNITS['Distance_mean']}]")
            plt.ylabel(f"{col} [{UNITS.get(col, '')}]")
            plt.grid(True, which='both', alpha=0.3)
            plt.legend()
            plt.tight_layout()
            plt.show()

# ----------------------------------------------------
# Run it
 # ----------------------------------------------------
redo_data_plots(MAG_df_wmean)


# Calculate deltas 

In [ ]:
SWA_df_wmean['delta_|V|'] = SWA_df_wmean['|V|'] - SWA_df_wmean['|V|_mean']
SWA_df_wmean['delta_VR'] = SWA_df_wmean['VR'] - SWA_df_wmean['VR_mean']
SWA_df_wmean['delta_VN'] = SWA_df_wmean['VN'] - SWA_df_wmean['VN_mean']
SWA_df_wmean['delta_VT'] = SWA_df_wmean['VT'] - SWA_df_wmean['VT_mean']
SWA_df_wmean['delta_N'] = SWA_df_wmean['N'] - SWA_df_wmean['N_mean']
SWA_df_wmean['delta_T_R'] = SWA_df_wmean['T_R'] - SWA_df_wmean['T_R_mean']
SWA_df_wmean['delta_T_T'] = SWA_df_wmean['T_T'] - SWA_df_wmean['T_T_mean']
SWA_df_wmean['delta_T_N'] = SWA_df_wmean['T_N'] - SWA_df_wmean['T_N_mean']
SWA_df_wmean['delta_T'] = SWA_df_wmean['T'] - SWA_df_wmean['T_mean']

SWA_df_wmean['delta_T_RR'] = SWA_df_wmean['T_RR'] - SWA_df_wmean['T_RR_mean']
SWA_df_wmean['delta_T_RT'] = SWA_df_wmean['T_RT'] - SWA_df_wmean['T_RT_mean']
SWA_df_wmean['delta_T_RN'] = SWA_df_wmean['T_RN'] - SWA_df_wmean['T_RN_mean']
SWA_df_wmean['delta_T_TT'] = SWA_df_wmean['T_TT'] - SWA_df_wmean['T_TT_mean']
SWA_df_wmean['delta_T_TN'] = SWA_df_wmean['T_TN'] - SWA_df_wmean['T_TN_mean']
SWA_df_wmean['delta_T_NN'] = SWA_df_wmean['T_NN'] - SWA_df_wmean['T_NN_mean']


MAG_df_wmean['delta_|B|'] = MAG_df_wmean['|B|'] - MAG_df_wmean['|B|_mean'] #check if this needs RMS
MAG_df_wmean['delta_BR'] = MAG_df_wmean['BR'] - MAG_df_wmean['BR_mean']
MAG_df_wmean['delta_BT'] = MAG_df_wmean['BT'] - MAG_df_wmean['BT_mean']
MAG_df_wmean['delta_BN'] = MAG_df_wmean['BN'] - MAG_df_wmean['BN_mean']
MAG_df_wmean['delta_|B_transv|'] = MAG_df_wmean['|B_transv|'] -MAG_df_wmean['|B_transv|_mean']
MAG_df_wmean['delta_B_parallel_0'] = MAG_df_wmean['B_//_0'] - MAG_df_wmean['B_parallel_0_mean'] #potential issue here?
MAG_df_wmean['delta_|B_perp_0|'] = MAG_df_wmean['|B_perp_0|'] #- MAG_df_wmean['|B_perp_0|_mean'] #inconsistent here? check if this needs RMS\






MAG_df_wmean['squared_delta_BR'] = MAG_df_wmean['delta_BR'] ** 2
MAG_df_wmean['squared_delta_BT'] = MAG_df_wmean['delta_BT'] ** 2
MAG_df_wmean['squared_delta_BN'] = MAG_df_wmean['delta_BN'] ** 2
MAG_df_wmean['squared_delta_|B|'] = MAG_df_wmean['delta_|B|'] ** 2
MAG_df_wmean['squared_delta_|B_transv|'] = MAG_df_wmean['delta_|B_transv|'] ** 2
MAG_df_wmean['squared_delta_B_parallel_0'] = MAG_df_wmean['delta_B_parallel_0'] ** 2
MAG_df_wmean['squared_delta_|B_perp_0|'] = MAG_df_wmean['delta_|B_perp_0|'] ** 2 #not needed?
MAG_df_wmean['squared_|delta_B_vector|'] = MAG_df_wmean['delta_BR']**2 + MAG_df_wmean['delta_BT']**2 + MAG_df_wmean['delta_BN']**2


SWA_df_wmean['squared_delta_|V|'] = SWA_df_wmean['delta_|V|'] ** 2
SWA_df_wmean['squared_delta_VR'] = SWA_df_wmean['delta_VR'] ** 2
SWA_df_wmean['squared_delta_VN'] = SWA_df_wmean['delta_VN'] ** 2
SWA_df_wmean['squared_delta_VT'] = SWA_df_wmean['delta_VT'] ** 2
SWA_df_wmean['squared_delta_N'] = SWA_df_wmean['delta_N'] ** 2
SWA_df_wmean['squared_delta_T_R'] = SWA_df_wmean['delta_T_R'] ** 2
SWA_df_wmean['squared_delta_T_T'] = SWA_df_wmean['delta_T_T'] ** 2
SWA_df_wmean['squared_delta_T_N'] = SWA_df_wmean['delta_T_N'] ** 2
SWA_df_wmean['squared_delta_T'] = SWA_df_wmean['delta_T'] ** 2
SWA_df_wmean['squared_delta_T_RR'] = SWA_df_wmean['delta_T_RR'] ** 2
SWA_df_wmean['squared_delta_T_RT'] = SWA_df_wmean['delta_T_RT'] ** 2
SWA_df_wmean['squared_delta_T_RN'] = SWA_df_wmean['delta_T_RN'] ** 2
SWA_df_wmean['squared_delta_T_TT'] = SWA_df_wmean['delta_T_TT'] ** 2
SWA_df_wmean['squared_delta_T_TN'] = SWA_df_wmean['delta_T_TN'] ** 2
SWA_df_wmean['squared_delta_T_NN'] = SWA_df_wmean['delta_T_NN'] ** 2




SWA_delta_means = SWA_df_wmean.groupby('name')[['delta_|V|','delta_VR','delta_VN','delta_VT','delta_N', 'squared_delta_|V|','squared_delta_VR','squared_delta_VN','squared_delta_VT', 'squared_delta_N','delta_T_R','delta_T_T','delta_T_N','delta_T','delta_T_RR','delta_T_RT','delta_T_RN','delta_T_TT','delta_T_TN','delta_T_NN']].mean()
MAG_delta_means = MAG_df_wmean.groupby('name')[['delta_|B|', 'squared_delta_BR', 'squared_delta_BT', 'squared_delta_BN', 'squared_delta_|B|','delta_|B_transv|','squared_delta_|B_transv|','squared_delta_B_parallel_0','squared_delta_|B_perp_0|','squared_|delta_B_vector|']].mean()
#new dataframe
all_delta_means = pd.merge(SWA_delta_means, MAG_delta_means, on='name', how='left')
all_delta_means.rename(columns=lambda x: "mean_"+x, inplace=True)
#new columns
all_delta_means['root_mean_squared_delta_BR'] = np.sqrt(all_delta_means['mean_squared_delta_BR'])
all_delta_means['root_mean_squared_delta_BT'] = np.sqrt(all_delta_means['mean_squared_delta_BT'])
all_delta_means['root_mean_squared_delta_BN'] = np.sqrt(all_delta_means['mean_squared_delta_BN'])
all_delta_means['root_mean_squared_delta_N'] = np.sqrt(all_delta_means['mean_squared_delta_N'])
all_delta_means['root_mean_squared_delta_|V|'] = np.sqrt(all_delta_means['mean_squared_delta_|V|'])
all_delta_means['root_mean_squared_delta_VR'] = np.sqrt(all_delta_means['mean_squared_delta_VR'])
all_delta_means['root_mean_squared_delta_VN'] = np.sqrt(all_delta_means['mean_squared_delta_VN'])
all_delta_means['root_mean_squared_delta_VT'] = np.sqrt(all_delta_means['mean_squared_delta_VT'])
all_delta_means['root_mean_squared_delta_|B|'] = np.sqrt(all_delta_means['mean_squared_delta_|B|'])
all_delta_means['root_mean_squared_delta_|B_transv|'] = np.sqrt(all_delta_means['mean_squared_delta_|B_transv|'])
all_delta_means['root_mean_squared_delta_B_parallel_0'] = np.sqrt(all_delta_means['mean_squared_delta_B_parallel_0'])
all_delta_means['root_mean_squared_delta_|B_perp_0|'] = np.sqrt(all_delta_means['mean_squared_delta_|B_perp_0|'])
all_delta_means['root_mean_squared_|delta_B_vector|'] = np.sqrt(all_delta_means['mean_squared_|delta_B_vector|'])



all_delta_means['Distance'] = mean_values_df['Distance_mean']
all_delta_means

In [ ]:
all_delta_means



In [ ]:
mean_values_df

In [ ]:
list(all_delta_means.columns)

FROM HERE 

In [ ]:
#all_delta_means.to_excel('all_delta_means.xlsx')

In [ ]:
list(mean_values_df.columns)

In [ ]:
list(MAG_df_wmean.columns)

In [ ]:
# Merge the all_delta_means DataFrame with the mean_values_df DataFrame on the 'name' column
all_delta_means_withmeans = pd.merge(all_delta_means, mean_values_df, on='name', how='inner')

# Display the merged DataFrame
all_delta_means_withmeans.head()

In [ ]:
# Create new columns in new dataframe
# Calculate delta_parallel and delta_perp assuming delta_parallel is along BR direction and delta_perp is perp to BR direction assuming mean field is mostly radial close to sun
all_delta_means_withmeans['root_mean_squared_delta_B_parallel'] = all_delta_means_withmeans['root_mean_squared_delta_BR']
all_delta_means_withmeans['root_mean_squared_delta_B_perp']=np.sqrt((all_delta_means_withmeans['root_mean_squared_delta_BT'])**2+(all_delta_means_withmeans['root_mean_squared_delta_BN'])**2)

# Calculate the ratios with perpendicular and parallel components
all_delta_means_withmeans['delta_parallel/delta_BT'] = (all_delta_means_withmeans['root_mean_squared_delta_B_parallel'])/(all_delta_means_withmeans['root_mean_squared_delta_BT'])
all_delta_means_withmeans['delta_parallel/delta_BN'] = (all_delta_means_withmeans['root_mean_squared_delta_B_parallel'])/(all_delta_means_withmeans['root_mean_squared_delta_BN'])

# Calculate the ratios with average B magnitude
all_delta_means_withmeans['delta_parallel/perp'] =(all_delta_means_withmeans['root_mean_squared_delta_B_parallel'])/(all_delta_means_withmeans['root_mean_squared_delta_B_perp'])
all_delta_means_withmeans['delta_perp/|B|'] =(all_delta_means_withmeans['root_mean_squared_delta_B_perp'])/(all_delta_means_withmeans['|B|_mean'])
all_delta_means_withmeans['delta_parallel/|B|'] =(all_delta_means_withmeans['root_mean_squared_delta_B_parallel'])/(all_delta_means_withmeans['|B|_mean'])
all_delta_means_withmeans['delta_|B|/|B|'] =(all_delta_means_withmeans['root_mean_squared_delta_|B|'])/(all_delta_means_withmeans['|B|_mean'])

# Calculate ratios with magntitude of average B vector= B_0
all_delta_means_withmeans['delta_perp/B_0'] =(all_delta_means_withmeans['root_mean_squared_delta_B_perp'])/(all_delta_means_withmeans['|mean_B_vector|'])
all_delta_means_withmeans['delta_parallel/B_0'] =(all_delta_means_withmeans['root_mean_squared_delta_B_parallel'])/(all_delta_means_withmeans['|mean_B_vector|'])
all_delta_means_withmeans['delta_|B|/B_0'] =(all_delta_means_withmeans['root_mean_squared_delta_|B|'])/(all_delta_means_withmeans['|mean_B_vector|'])

# Calculate ratios with magntitude of average B vector= B_0 where  perpendicular and parallel components are defined wrt B_0
all_delta_means_withmeans['delta_B_parallel_0/B_0'] =(all_delta_means_withmeans['root_mean_squared_delta_B_parallel_0'])/(all_delta_means_withmeans['|mean_B_vector|'])

all_delta_means_withmeans['delta_B_parallel_0'] =(all_delta_means_withmeans['root_mean_squared_delta_B_parallel_0'])
all_delta_means_withmeans['delta_B_perp_0'] =(all_delta_means_withmeans['root_mean_squared_delta_|B_perp_0|'])

all_delta_means_withmeans['delta_B_perp_0/B_0'] =(all_delta_means_withmeans['delta_B_perp_0'])/(all_delta_means_withmeans['|mean_B_vector|'])

all_delta_means_withmeans['delta_B_parallel_0/delta_B_perp_0'] =(all_delta_means_withmeans['root_mean_squared_delta_B_parallel_0'])/(all_delta_means_withmeans['delta_B_perp_0'])




#Sanity check if deltas are correct

all_delta_means_withmeans['deltas_RTN']=(all_delta_means_withmeans['root_mean_squared_delta_BR'])**2+(all_delta_means_withmeans['root_mean_squared_delta_BT'])**2+(all_delta_means_withmeans['root_mean_squared_delta_BN'])**2

all_delta_means_withmeans['deltas_RTN_check']=(all_delta_means_withmeans['root_mean_squared_delta_B_parallel'])**2 + (all_delta_means_withmeans['root_mean_squared_delta_B_perp'])**2



all_delta_means_withmeans['deltas_//†']=(all_delta_means_withmeans['root_mean_squared_delta_B_parallel_0'])**2+(all_delta_means_withmeans['root_mean_squared_delta_|B_perp_0|'])**2

all_delta_means_withmeans['deltas_//†_check']=(all_delta_means_withmeans['root_mean_squared_|delta_B_vector|'])**2

#check 2 quantities are equal
all_delta_means_withmeans['deltas_difference_check']=(all_delta_means_withmeans['deltas_//†_check'])- all_delta_means_withmeans['deltas_RTN_check']

print(all_delta_means_withmeans[['deltas_//†_check','deltas_RTN_check','deltas_difference_check' ]])


In [ ]:


#Sphere relation check
all_delta_means_withmeans['sphere_relation_RHS']=(all_delta_means_withmeans['delta_B_parallel_0'])

all_delta_means_withmeans['sphere_relation_LHS']=((all_delta_means_withmeans['delta_B_perp_0']))**2/(2*(all_delta_means_withmeans['|mean_B_vector|']))

print(all_delta_means_withmeans['sphere_relation_RHS']/all_delta_means_withmeans['sphere_relation_LHS'])


all_delta_means_withmeans


# Parker Spiral Angles 

In [ ]:
# Angle between the background magnetic field B0 and the radial direction,
# computed using |BR| so the angle is independent of field polarity
# independent of whether the field points outward or inward

all_delta_means_withmeans['cos_theta_PS'] = (all_delta_means_withmeans['BR_mean'].abs() / all_delta_means_withmeans['|mean_B_vector|'])

# numerical safety
#cos_theta = cos_theta_axis.clip(-1, 1)

all_delta_means_withmeans['theta_PS_observed'] = np.degrees(np.arccos(all_delta_means_withmeans['cos_theta_PS']))



# Angle between the background magnetic field B0 and the tangential direction,
# computed using |BT_mean| so the angle is independent of polarity
all_delta_means_withmeans['phi_PS'] = 90 - all_delta_means_withmeans['theta_PS_observed']


all_delta_means_withmeans['theta_PS_theory']= np.degrees(np.arctan(all_delta_means_withmeans['Distance_mean']*AU_M*2.865e-6/ ((all_delta_means_withmeans['|V|_mean'])*KM_TO_M))) #Parker spiral angle theory
print(AU_M)
display(all_delta_means_withmeans[['theta_PS_theory', 'theta_PS_observed','BR_mean','|mean_B_vector|']])
MAG_df_wmean[['theta_PS_observed','name','BR_mean','|mean_B_vector|']].drop_duplicates()

In [ ]:
all_delta_means_withmeans.columns

# plots for parker spiral angles

In [ ]:
#  Radial Scalling of B_PARALLEL_0_mean AND |B_PERP_0|_mean and Parker Spiral Angles

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress


UNITS = {'Distance_mean': 'AU','theta_PS_observed': 'deg','theta_PS_theory': 'deg'}

# ----------------------------------------------------
# Main plotting function
# ----------------------------------------------------
def redo_data_plots(df):

    columns = ['theta_PS_observed','theta_PS_theory']
        

    for col in columns:

        x = df['Distance_mean'].values
        y = df[col].values

        # =========================================================
        # 1) LINEAR PLOT
        # =========================================================
        mask_lin = np.isfinite(x) & np.isfinite(y)
        if mask_lin.sum() < 2:
            continue

        x_lin = x[mask_lin]
        y_lin = y[mask_lin]

        slope_lin, intercept_lin, _, _, _ = linregress(x_lin, y_lin)

        x_fit_lin = np.linspace(x_lin.min(), x_lin.max(), 200)
        y_fit_lin = slope_lin * x_fit_lin + intercept_lin

        plt.figure()
        sns.scatterplot(x=x_lin, y=y_lin, label=col,color='darkred' if col=='theta_PS_observed' else 'darkblue')
        plt.plot(x_fit_lin, y_fit_lin,'r--' if col=='theta_PS_observed' else 'b--',
                 label=f'Linear Fit (slope = {slope_lin:.2f})')

        plt.title(f'{col} vs Distance')
        plt.xlabel(f"Distance [{UNITS['Distance_mean']}]")
        plt.ylabel(f"{col} [{UNITS.get(col, '')}]")
        plt.grid(True, which='both', alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()

# # =========================================================
    x = df['Distance_mean'].values
    y1 = df['theta_PS_observed'].values
    y2= df['theta_PS_theory'].values

    # =========================================================
    # 1) LINEAR PLOT
    # =========================================================
  
    x_lin = x
    y1_lin = y1
    y2_lin = y2

    slope_lin1, intercept_lin1, _, _, _ = linregress(x_lin, y1_lin)
    slope_lin2, intercept_lin2, _, _, _ = linregress(x_lin, y2_lin)


    x_fit_lin = np.linspace(x_lin.min(), x_lin.max(), 200)
    y1_fit_lin = slope_lin1 * x_fit_lin + intercept_lin1
    y2_fit_lin = slope_lin2 * x_fit_lin + intercept_lin2

    plt.figure()
    sns.scatterplot(x=x_lin, y=y1_lin, label='theta_PS_observed',color='darkred')
    sns.scatterplot(x=x_lin, y=y2_lin, label='theta_PS_theory',color='darkblue')
    plt.plot(x_fit_lin, y1_fit_lin,'r--',
             label=f'Linear Fit (slope = {slope_lin1:.2f})')
    plt.plot(x_fit_lin, y2_fit_lin,'b--',
             label=f'Linear Fit (slope = {slope_lin2:.2f})')

    plt.title(f'{col} vs Distance')
    plt.xlabel(f"Distance [{UNITS['Distance_mean']}]")
    plt.ylabel("theta [deg]")
    plt.grid(True, which='both', alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
redo_data_plots(all_delta_means_withmeans)


# plots for deltas



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

# ----------------- Units -----------------
UNITS_deltas = {
    # Velocities
    'mean_delta_VR': 'km/s', 'mean_delta_VN': 'km/s', 'mean_delta_VT': 'km/s',
    'mean_delta_|V|': 'km/s',
    'root_mean_squared_delta_VR': 'km/s', 'root_mean_squared_delta_VN': 'km/s',
    'root_mean_squared_delta_VT': 'km/s', 'root_mean_squared_delta_|V|': 'km/s',

    # Density
    'mean_delta_N': r'cm$^{-3}$', 'root_mean_squared_delta_N': r'cm$^{-3}$',

    # Distance
    'Distance': 'AU',

    # Temperatures (scalar + tensor)
    'mean_delta_T_R': 'K',
    'mean_delta_T_T': 'K',
    'mean_delta_T_N': 'K',
    'mean_delta_T': 'K',
    'mean_delta_T_RR': 'K',
    'mean_delta_T_RT': 'K',
    'mean_delta_T_RN': 'K',
    'mean_delta_T_TT': 'K',
    'mean_delta_T_TN': 'K',
    'mean_delta_T_NN': 'K',

    # Magnetic field
    'mean_delta_|B|': 'nT',
    'root_mean_squared_delta_|B|': 'nT',
    'root_mean_squared_delta_BR': 'nT',
    'root_mean_squared_delta_BT': 'nT',
    'root_mean_squared_delta_BN': 'nT',
    'mean_delta_|B_transv|': 'nT',
    'root_mean_squared_delta_|B_transv|': 'nT'
}

def sort_xy(x, y):
    x = np.asarray(x)
    y = np.asarray(y)
    order = np.argsort(x)
    return x[order], y[order]


# ----------------- Main plotting function -----------------
def redo_all_delta_means_plots(df):

    columns = [
        'mean_delta_VR', 'root_mean_squared_delta_VR', 'mean_delta_VN', 'root_mean_squared_delta_VN',
        'mean_delta_VT', 'root_mean_squared_delta_VT', 'mean_delta_|V|', 'root_mean_squared_delta_|V|',
        'mean_delta_N', 'root_mean_squared_delta_N', 'mean_delta_|B|', 'root_mean_squared_delta_|B|',
        'root_mean_squared_delta_BR', 'root_mean_squared_delta_BT', 'root_mean_squared_delta_BN',
        'root_mean_squared_delta_|B_transv|', 'mean_delta_|B_transv|',
        'mean_delta_T_R', 'mean_delta_T_T', 'mean_delta_T_N', 'mean_delta_T',
        'mean_delta_T_RR', 'mean_delta_T_RT', 'mean_delta_T_RN',
        'mean_delta_T_TT', 'mean_delta_T_TN', 'mean_delta_T_NN',
        'delta_parallel/perp', 'delta_perp/|B|', 'delta_parallel/|B|', 'delta_|B|/|B|',
        'delta_perp/B_0', 'delta_parallel/B_0', 'delta_|B|/B_0',
        'delta_parallel/delta_BT', 'delta_parallel/delta_BN',
        'delta_B_parallel_0/B_0', 'delta_B_perp_0/B_0','delta_B_parallel_0/delta_B_perp_0','delta_B_parallel_0', 'delta_B_perp_0'
    ]

    # Columns that ALSO get a log–log + power-law fit
    cols_to_log = [
        'root_mean_squared_delta_BR', 'root_mean_squared_delta_BT', 'root_mean_squared_delta_BN',
        'root_mean_squared_delta_N', 'root_mean_squared_delta_|V|', 'root_mean_squared_delta_|B|',
        'root_mean_squared_delta_|B_transv|', 'root_mean_squared_delta_VR', 'root_mean_squared_delta_VN',
        'root_mean_squared_delta_VT', 'root_mean_squared_delta_|B_transv|',
        'delta_parallel/perp', 'delta_perp/|B|', 'delta_parallel/|B|', 'delta_|B|/|B|',
        'delta_parallel/delta_BT', 'delta_parallel/delta_BN',
        'delta_perp/B_0', 'delta_parallel/B_0', 'delta_|B|/B_0',
        'delta_B_parallel_0/B_0', 'delta_B_perp_0/B_0','delta_B_parallel_0/delta_B_perp_0', 'delta_B_parallel_0', 'delta_B_perp_0'
    ]

    for col in columns:
        if col not in df.columns:
            continue  # skip if column not present

        x = df['Distance'].values
        y = df[col].values

        # ---------------- 1) Linear plot + linear fit ----------------
        mask_lin = np.isfinite(x) & np.isfinite(y)
        if mask_lin.sum() < 2:
            continue

        x_lin = x[mask_lin]
        y_lin = y[mask_lin]

        slope_lin, intercept_lin, _, _, _ = linregress(x_lin, y_lin)

        x_fit_lin = np.linspace(x_lin.min(), x_lin.max(), 200)
        y_fit_lin = slope_lin * x_fit_lin + intercept_lin

        plt.figure()
        sns.scatterplot(x=x_lin, y=y_lin, label=col)
        plt.plot(x_fit_lin, y_fit_lin, 'r--',
                 label=f'Linear Fit (slope = {slope_lin:.2f})')

        plt.title(f'{col} vs Distance')
        plt.xlabel(f"Distance [{UNITS_deltas['Distance']}]")

        if col in UNITS_deltas:
            ylabelstring = f"{col} [{UNITS_deltas[col]}]"
        else:
            ylabelstring = col
        plt.ylabel(ylabelstring)

        plt.grid(True, which='both', alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()

        # ---------------- 2) Log–log plot + power-law fit ----------------
        if col not in cols_to_log:
            continue

        mask_log = (x > 0) & (y > 0) & np.isfinite(x) & np.isfinite(y)
        if mask_log.sum() < 2:
            continue

        x_log = x[mask_log]
        y_log = y[mask_log]

        logx = np.log10(x_log)
        logy = np.log10(y_log)

        slope_pow, intercept_pow, _, _, _ = linregress(logx, logy)

        # y = 10^intercept_pow * x^slope_pow
        x_fit_log = np.logspace(np.log10(x_log.min()),
                                np.log10(x_log.max()), 200)
        y_fit_log = 10**intercept_pow * x_fit_log**slope_pow

        plt.figure()
        sns.scatterplot(x=x_log, y=y_log, label=col)
        plt.plot(x_fit_log, y_fit_log, 'r--',
                 label=f'Power-law Fit (slope = {slope_pow:.2f})')

        plt.xscale('log')
        plt.yscale('log')

        # Force y-axis to span at least 1 decade (=> at least 2 round log numbers)
        ymin = y_log.min()
        ymax = y_log.max()

        log_ymin = np.floor(np.log10(ymin))
        log_ymax = np.ceil(np.log10(ymax))

        if (log_ymax - log_ymin) < 1:
            # expand symmetrically around the middle in log space
            log_ymid = 0.5 * (log_ymin + log_ymax)
            log_ymin = np.floor(log_ymid - 0.5)
            log_ymax = np.ceil(log_ymid + 0.5)

        plt.ylim(10**log_ymin, 10**log_ymax)

        plt.title(f'{col} vs Distance (log–log scale)')
        plt.xlabel(f"Distance [{UNITS_deltas['Distance']}]")
        plt.ylabel(ylabelstring)
        plt.grid(True, which='both', alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()


# Example call:
redo_all_delta_means_plots(all_delta_means_withmeans)
# redo_all_delta_means_plots(positive_all_delta_means)
# redo_all_delta_means_plots(negative_all_delta_means)


PLOTS FOR DELTA EQUATIONS

In [ ]:
def redo_all_delta_means_plots(df):

    # We now only plot these sanity-check pairs:
    pairs = [
        ('deltas_RTN', 'deltas_RTN_check'),
        ('deltas_//†', 'deltas_//†_check'),
    ]

    # --------- Pretty axis labels ---------
    LABELS = {
        'deltas_RTN'        : r'$<(\delta B_R)>^2 + <(\delta B_T)>^2 + <(\delta B_N)>^2$',
        'deltas_RTN_check'  : r'$<(\delta B_{\parallel})>^2 + <(\delta B_{\perp})>^2$',

        'deltas_//†'        : r'$<(\delta B_{\parallel,0})>^2 + <|\delta B_{\perp,0}|>^2$',
        'deltas_//†_check'  : r'$<|\delta \mathbf{B}|>^2$'
    }

    for x_col, y_col in pairs:
        if (x_col not in df.columns) or (y_col not in df.columns):
            continue

        x = df[x_col].values
        y = df[y_col].values

        # ---------------- 1) Linear plot + linear fit ----------------
        mask_lin = np.isfinite(x) & np.isfinite(y)
        if mask_lin.sum() < 2:
            continue

        x_lin = x[mask_lin]
        y_lin = y[mask_lin]

        # sort
        x_lin, y_lin = sort_xy(x_lin, y_lin)

        slope_lin, intercept_lin, _, _, _ = linregress(x_lin, y_lin)

        x_fit_lin = np.linspace(x_lin.min(), x_lin.max(), 200)
        y_fit_lin = slope_lin * x_fit_lin + intercept_lin

# --------- HIGH-PRECISION PRINTS ---------


        plt.figure()
        sns.scatterplot(x=x_lin, y=y_lin)
        plt.plot(x_fit_lin, y_fit_lin, 'r--',label=f'Linear Fit (slope = {slope_lin:.10f}, intercept = {intercept_lin:.10e})')


        # y = x reference line
        min_val = min(x_lin.min(), y_lin.min())
        max_val = max(x_lin.max(), y_lin.max())
        plt.plot([min_val, max_val], [min_val, max_val], 'k:',
                 label='y = x')

        # --------- USE PRETTY LABELS HERE ---------
        plt.xlabel(LABELS[x_col])
        plt.ylabel(LABELS[y_col])

        plt.title(f'{LABELS[y_col]} vs {LABELS[x_col]}')

        plt.grid(True, which='both', alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()


redo_all_delta_means_plots(all_delta_means_withmeans)

## sphere relation check plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

# Helper if not defined
def sort_xy(x, y):
    idx = np.argsort(x)
    return x[idx], y[idx]

def plot_sphere_relation(df):
    """
    Plots sphere_relation_LHS (x) vs sphere_relation_RHS (y)
    Linear scale + log–log scale.
    """

    # Fixed column names --------------------------------
    x_col = 'sphere_relation_LHS'
    y_col = 'sphere_relation_RHS'

    # Pretty labels -------------------------------------
    LABELS = {'sphere_relation_RHS': r'$\langle \delta B_{\parallel} \rangle$',
              'sphere_relation_LHS': r'$\langle \delta B_{\perp} \rangle^{2} / (2\,B_{0})$'}


    # Check existence -----------------------------------
    if (x_col not in df.columns) or (y_col not in df.columns):
        print("Required columns missing in dataframe.")
        return

    x = df[x_col].values
    y = df[y_col].values

    # ===================== 1) LINEAR PLOT =======================
    mask_lin = np.isfinite(x) & np.isfinite(y)
    if mask_lin.sum() < 2:
        print("Not enough valid points.")
        return

    x_lin, y_lin = sort_xy(x[mask_lin], y[mask_lin])

    slope_lin, intercept_lin, _, _, _ = linregress(x_lin, y_lin)

    x_fit = np.linspace(x_lin.min(), x_lin.max(), 200)
    y_fit = slope_lin * x_fit + intercept_lin

    plt.figure()
    sns.scatterplot(x=x_lin, y=y_lin)
    plt.plot(x_fit, y_fit, 'r--',
             label=f'Linear Fit (slope = {slope_lin:.10f}, intercept = {intercept_lin:.4e})')

    # # y = x line
    # min_val = min(x_lin.min(), y_lin.min())
    # max_val = max(x_lin.max(), y_lin.max())
    # plt.plot([min_val, max_val], [min_val, max_val], 'k:', label='y = x')

    xlabel = LABELS[x_col]
    ylabel = LABELS[y_col]

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(f'{ylabel} vs {xlabel}')

    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # ===================== 2) LOG–LOG PLOT =======================
    mask_log = (x > 0) & (y > 0) & np.isfinite(x) & np.isfinite(y)
    if mask_log.sum() < 2:
        print("Not enough positive values for log–log.")
        return

    x_log, y_log = sort_xy(x[mask_log], y[mask_log])

    logx = np.log10(x_log)
    logy = np.log10(y_log)

    slope_pow, intercept_pow, _, _, _ = linregress(logx, logy)

    x_fit_log = np.logspace(np.log10(x_log.min()), np.log10(x_log.max()), 200)
    y_fit_log = 10**intercept_pow * x_fit_log**slope_pow

    # # ----- y = x reference line (log–log safe) -----
    # min_val = min(x_log.min(), y_log.min())
    # max_val = max(x_log.max(), y_log.max())

    # # Generate a smooth diagonal line in log–log space
    # ref_x = np.logspace(np.log10(min_val), np.log10(max_val), 200)
    # ref_y = ref_x.copy()



    plt.figure()
    sns.scatterplot(x=x_log, y=y_log, label=f'{ylabel} vs {xlabel}')
    # plt.plot(ref_x, ref_y, 'k:', linewidth=1, label='y = x')
    plt.plot(x_fit_log, y_fit_log, 'r--', label=f'Power-law slope = {slope_pow:.4f}')

    plt.xscale('log')
    plt.yscale('log')

    # y-axis at least 1 decade
    log_ymin = np.floor(logy.min())
    log_ymax = np.ceil(logy.max())

    if log_ymax - log_ymin < 1:
        mid = 0.5 * (log_ymin + log_ymax)
        log_ymin = np.floor(mid - 0.5)
        log_ymax = np.ceil(mid + 0.5)

    plt.ylim(10**log_ymin, 10**log_ymax)

    plt.title(f'{ylabel} vs {xlabel} (log–log)')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()


plot_sphere_relation(all_delta_means_withmeans)


# Parker Spiral angle plot


In [ ]:
# # Mass Flux Plots
# # Loop through each date range in the daterange dictionary
# for name in SWA_df['name'].unique():
#                         # Get the name of the range
#
#     # Filter the dataframes for the current date range
#     SWA_filtered = SWA_df[SWA_df['name'] == name]
#
#
#     # Create subplots for plotting                       1st number is the width, 2nd is the height
#     fig, ax = plt.subplots(1, 1, layout='tight', sharex=True, figsize=(8, 6))
#
#     # Plotting for Proton Velocity Moments
#     ax.set_title(f'Mass Flux: {name}')
#     ax.plot(SWA_filtered['mass_flux'], color='red', label='Mass Flux')# ,linewidth=0.2)
#     ax.tick_params(top=False, bottom=True, left=True, right=False, labelbottom=True, labeltop=False, labelleft=True, labelright=False)
#     ax.legend()
#
#
#     # Format the dates nicely
#     fig.autofmt_xdate()
#
#     # Show each figure
#     # fig.show()
#     plotly_fig = tls.mpl_to_plotly(fig)
#     legend = go.layout.Legend(
#         x=1,
#         y=0.5
#     )
#     plotly_fig.update_layout(showlegend=True,legend=legend)
#     plotly_fig.update_xaxes(matches='x')
#     iplot(plotly_fig)

In [ ]:
mean_values_df

In [ ]:
all_mean_table=pd.merge(mean_values_df, all_delta_means, on='name')
all_mean_table

In [ ]:
all_mean_table['average_delta_BR_star']=(all_mean_table['mean_delta_|B_transv|']**2)/(all_mean_table['|B|_mean']*2)
all_mean_table['average_delta_BR_star_TRIAL']=(((all_mean_table['root_mean_squared_delta_BN']**2)+(all_mean_table['root_mean_squared_delta_BT']**2))/(all_mean_table['|mean_B_vector|']*2))*1.4

In [ ]:
display(all_mean_table[['average_delta_BR_star','average_delta_BR_star_TRIAL','root_mean_squared_delta_BR']])

all_mean_table[['mean_delta_|B_transv|', 'root_mean_squared_delta_BN']]


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
#plt.scatter(all_mean_table['Distance'], all_mean_table['average_delta_BR_star'], label='average_delta_BR_star', marker='o')
plt.scatter(all_mean_table['Distance'], all_mean_table['average_delta_BR_star_TRIAL'], label='average_delta_BR_star_TRIAL [nT]', marker='x')
plt.scatter(all_mean_table['Distance'], all_mean_table['root_mean_squared_delta_BR'], label='root_mean_squared_delta_BR [nT]', marker='s')

plt.xlabel('Distance [AU]')
plt.ylabel('average_delta_BR [nT]')
plt.title('average_delta_BR vs Distance')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Split all_mean_table into two groups based on the sign of BR_mean
positive_all_mean_table = all_mean_table[all_mean_table['BR_mean'] > 0]
negative_all_mean_table = all_mean_table[all_mean_table['BR_mean'] < 0]

# Function to redo the 'average_delta_BR vs Distance' plots for a given DataFrame
def redo_average_delta_BR_plots(df, group_name):
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 6))
    plt.scatter(df['Distance'], df['average_delta_BR_star_TRIAL'], label='average_delta_BR_star_TRIAL [nT]', marker='x')
    plt.scatter(df['Distance'], df['root_mean_squared_delta_BR'], label='root_mean_squared_delta_BR [nT]', marker='s')

    plt.xlabel('Distance [AU]')
    plt.ylabel('average_delta_BR [nT]')
    plt.title(f'average_delta_BR vs Distance ({group_name})')
    plt.legend()
    plt.grid()
    plt.show()

# Redo the 'average_delta_BR vs Distance' plots for positive BR_mean
redo_average_delta_BR_plots(positive_all_mean_table, 'Positive BR_mean')

# Redo the 'average_delta_BR vs Distance' plots for negative BR_mean
redo_average_delta_BR_plots(negative_all_mean_table, 'Negative BR_mean')

In [ ]:
MAG_df_wmean

In [ ]:
MAG_df_wmean['delta_BR_star']=(MAG_df_wmean['delta_|B_transv|']**2)/(MAG_df_wmean['|B|']*2)
MAG_df_wmean['delta_BR_star_TRIAL']=((MAG_df_wmean['delta_BN']**2)+(MAG_df_wmean['delta_BT']**2))/(MAG_df_wmean['|B|']*2)

In [ ]:
MAG_df_wmean[['delta_BR_star','delta_BR_star_TRIAL','delta_BR']]

In [ ]:
# # Plot delta_BR_star, delta_BR_star_TRIAL, and delta_BR vs EPOCH for each 'name' group interactively
# 
# import matplotlib.pyplot as plt
# import pandas as pd
# 
# # Ensure the 'name' column exists in the DataFrame and 'EPOCH' is in a proper datetime format
# for name, group in MAG_df_wmean.groupby('name'):
#     fig, ax = plt.subplots(1, 1, layout='tight', sharex=True, figsize=(8, 6))
#     ax.plot(group['EPOCH'], group['delta_BR_star'], label='delta_BR_star',linestyle='-',linewidth=1)
#     ax.plot(group['EPOCH'], group['delta_BR_star_TRIAL'], label='delta_BR_star_TRIAL', linestyle='-',linewidth=1)
#     ax.plot(group['EPOCH'], (group['delta_BR']), label='delta_BR', linestyle='-',linewidth=1)
# 
# 
#     ax.set_title(f'delta_BR vs EPOCH for {name}')
#     ax.legend()
#     fig.autofmt_xdate()
# 
#     # Show each figure
#     # fig.show()
#     plotly_fig = tls.mpl_to_plotly(fig)
#     legend = go.layout.Legend(
#         x=1,
#         y=0.5
#     )
#     plotly_fig.update_layout(showlegend=True,legend=legend)
#     plotly_fig.update_xaxes(matches='x')
#     iplot(plotly_fig)

In [ ]:
print(len(SWA_df), len(MAG_df))

SWA_df

In [ ]:
import numpy as np

mean_values_df_with_calcs = mean_values_df.copy()
#T is average T # possible issue here, T is interpolated but so are its sub components, should we do that or re calculate T based off the new interpolated sub components?

print(mean_values_df_with_calcs.columns)
#mean_values_df_with_calcs['|B|'] = np.linalg.norm(mean_values_df_with_calcs[['BR',"BT",'BN']], axis=1)
mean_values_df_with_calcs


##

In [ ]:
# @TODO check maths for T parallel and T perp
# TORA TO SWA_df_with_interpolated_MAG EXI KAI TA MAGNETIC FIELDS, SO MPORIS NA KANIS RUN TO CALCULATLATION
#T_||=T_R*B_R^2/|B|^2 + T_T*B_T^2/|B|^2 + T_N*B_N^2/|B|^2
mean_values_df_with_calcs['T_PARALLEL'] = ((mean_values_df_with_calcs['T_R_mean']* (mean_values_df_with_calcs['BR_mean'])**2) +  (mean_values_df_with_calcs['T_T_mean']* (mean_values_df_with_calcs['BT_mean'])**2) + (mean_values_df_with_calcs['T_N_mean']* (mean_values_df_with_calcs['BN_mean'])**2))/(mean_values_df_with_calcs['|B|_mean'])**2

mean_values_df_with_calcs['T_PARALLEL_fromPT'] =((mean_values_df_with_calcs['T_RR_mean'] * (mean_values_df_with_calcs['BR_mean']**2)) +
        (mean_values_df_with_calcs['T_TT_mean'] * (mean_values_df_with_calcs['BT_mean']**2)) +
        (mean_values_df_with_calcs['T_NN_mean'] * (mean_values_df_with_calcs['BN_mean']**2)) +
        (2 * mean_values_df_with_calcs['T_RT_mean'] * mean_values_df_with_calcs['BR_mean'] * mean_values_df_with_calcs['BT_mean']) +
        (2 * mean_values_df_with_calcs['T_RN_mean'] * mean_values_df_with_calcs['BR_mean'] * mean_values_df_with_calcs['BN_mean']) +
        (2 * mean_values_df_with_calcs['T_TN_mean'] * mean_values_df_with_calcs['BT_mean'] * mean_values_df_with_calcs['BN_mean']))/(mean_values_df_with_calcs['|B|_mean'])**2


# get T_perp by subtraction T_|| from the total T (trace)
mean_values_df_with_calcs['T_PERPENDICULAR']=1/2*(3*(mean_values_df_with_calcs['T_mean'])-mean_values_df_with_calcs['T_PARALLEL'])

# is Tmean the same as before where i just have a diagonal vector?
mean_values_df_with_calcs['T_PERPENDICULAR_fromPT']=1/2*(3*(mean_values_df_with_calcs['T_mean'])-mean_values_df_with_calcs['T_PARALLEL_fromPT'])

#T_PERP=T_R*(1-B_R^2/|B|^2) + T_T*(1-B_T^2/|B|^2) + T_N*(1-B_N^2/|B|^2)
def check_T_perpendicular(row):
    T_vector = np.array([row.T_R_mean, row.T_T_mean, row.T_N_mean])
    B_vector = np.array([(row.BR_mean)**2,(row.BT_mean)**2,(row.BN_mean)**2])
    B_vector = B_vector / (row["|B|_mean"])**2
    B_vector = (1 - B_vector)/2
    return np.dot(T_vector, B_vector)

def check_T_perpendicular_PT(row):
    T_tensor = np.array([
            [row.T_RR_mean, row.T_RT_mean, row.T_RN_mean],
            [row.T_RT_mean, row.T_TT_mean, row.T_TN_mean],
            [row.T_RN_mean, row.T_TN_mean, row.T_NN_mean]
        ])
    B_vector = np.array([(row.BR_mean)**2,(row.BT_mean)**2,(row.BN_mean)**2])
    B_vector = B_vector / (row["|B|_mean"])**2
    B_vector = (1 - B_vector)/2
    return np.dot(T_tensor, B_vector)

def check_T_3(row):
    # Normalize magnetic field to get unit vector b_hat
    B_vec = np.array([row.BR_mean,row.BT_mean,row.BN_mean])
    b_hat = B_vec / row["|B|_mean"]

    # Construct diagonal temperature tensor in RTN
    T_tensor = np.diag([row.T_R_mean, row.T_T_mean, row.T_N_mean])

    # Compute T_parallel = b̂ᵀ · T_tensor · b̂
    T_parallel = b_hat @ T_tensor @ b_hat

    #Compute T_perp assuming gyrotropy (T_perp is average of other two components)
    T_trace = np.trace(T_tensor)
    T_perp = 0.5 * (T_trace - T_parallel)
    return( T_trace,T_trace/3,T_parallel,T_perp)

def check_T_3_PT(row):
       B_vec = np.array([row.BR_mean,row.BT_mean,row.BN_mean])
       b_hat = B_vec / row["|B|_mean"]
       T_tensor = np.array([
            [row.T_RR_mean, row.T_RT_mean, row.T_RN_mean],
            [row.T_RT_mean, row.T_TT_mean, row.T_TN_mean],
            [row.T_RN_mean, row.T_TN_mean, row.T_NN_mean]
        ])
        # Compute T_parallel = b̂ᵀ · T_tensor · b̂
       T_parallel = b_hat @ T_tensor @ b_hat

        #Compute T_perp assuming gyrotropy (T_perp is average of other two components)
       T_trace = np.trace(T_tensor)
       T_perp = 0.5 * (T_trace - T_parallel)
       return( T_trace,T_trace/3,T_parallel,T_perp)




mean_values_df_with_calcs['T_PERPENDICULAR_2']=mean_values_df_with_calcs.apply(check_T_perpendicular, axis=1)
mean_values_df_with_calcs[['T_trace3','T_3','T_parallel3','T_perp3']]=mean_values_df_with_calcs.apply(check_T_3, axis=1, result_type="expand")

mean_values_df_with_calcs['T_PERPENDICULAR_2_PT']=mean_values_df_with_calcs.apply(check_T_perpendicular_PT, axis=1)
mean_values_df_with_calcs[['T_trace3_PT','T_3_PT','T_parallel3_PT','T_perp3_PT']]=mean_values_df_with_calcs.apply(check_T_3_PT, axis=1, result_type="expand")


mean_values_df_with_calcs


In [ ]:
mean_values_df_with_calcs['magnetic_moment']=mean_values_df_with_calcs['T_perp3']/mean_values_df_with_calcs['|B|_mean']
mean_values_df_with_calcs['second_adiabatic_invariant']=(mean_values_df_with_calcs['T_parallel3']*((mean_values_df_with_calcs['|B|_mean'])**2))/((mean_values_df_with_calcs['N_mean'])**2)


In [ ]:
# mean_values_df_with_calcs_means = mean_values_df_with_calcs.groupby('name').mean()
# mean_values_df_with_calcs_means

In [ ]:
# mean_values_df_with_calcs.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import linregress


# Define the quantities to plot
quantities = [
    'T_mean','T_parallel3', 'T_perp3','magnetic_moment',
    'second_adiabatic_invariant', '|B|_mean',
    "N_mean",'T_mean','T_R_mean','T_T_mean','T_N_mean'
]

# ---- Units for these quantities ----
UNITS_CALC = {
    'Distance_mean': 'AU',

    # Temperatures
    'T_mean': 'K',
    'T_R_mean': 'K', 'T_T_mean': 'K', 'T_N_mean': 'K',
    'T_parallel3': 'K', 'T_perp3': 'K',

    # Plasma / field
    'N_mean': r'cm$^{-3}$',
    '|B|_mean': 'nT',

    # Invariants
    'magnetic_moment': 'K/nT',
    'second_adiabatic_invariant': r'K nT$^{2}$ cm$^{-6}$'
}


# Loop through each quantity and generate a separate plot
for quantity in quantities:
    # Extract x (Distance) and y (current quantity)
    x = mean_values_df_with_calcs['Distance_mean'].values
    y = mean_values_df_with_calcs[quantity].values

    # Filter out NaN and infinite values
    mask = np.isfinite(x) & np.isfinite(y)
    x = x[mask]
    y = y[mask]

    if len(x) < 2:
        continue

    # -------- SORT BY x FOR NICE PLOTTING --------
    sort_idx = np.argsort(x)
    x = x[sort_idx]
    y = y[sort_idx]

    # Perform linear regression
    slope, intercept, _, _, _ = linregress(x, y)
    best_fit_line = slope * x + intercept

    # Create the plot
    plt.figure(figsize=(8,6))
    sns.scatterplot(x=x, y=y, s=50, alpha=0.7, color='darkblue', label=quantity)
    plt.plot(x, best_fit_line,
             color='darkred', linestyle='--',
             label=f'Best Fit Line (slope = {slope:.2f})')

    plt.title(f'{quantity} vs Distance')
    plt.xlabel('Distance [AU]')
    plt.ylabel(f"{quantity} [{UNITS_CALC.get(quantity, '')}]")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

    # Show the plot
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import linregress

# Quantities to plot (log–log + power-law fit)
quantities = [
    'T_mean','T_parallel3', 'T_perp3',
    'magnetic_moment','second_adiabatic_invariant',
    '|B|_mean', "N_mean",
    'T_mean','T_R_mean','T_T_mean','T_N_mean'
]

# (optional) units dict if you want proper y-labels
UNITS_CALC = {
    'Distance_mean': 'AU',
    'T_mean': 'K',
    'T_R_mean': 'K', 'T_T_mean': 'K', 'T_N_mean': 'K',
    'T_parallel3': 'K', 'T_perp3': 'K',
    'N_mean': r'cm$^{-3}$',
    '|B|_mean': 'nT',
    'magnetic_moment': 'K/nT',
    'second_adiabatic_invariant': r'K nT$^{2}$ cm$^{-6}$'
}

for quantity in quantities:
    print(quantity)

    # original (physical) values
    x = mean_values_df_with_calcs['Distance_mean'].values
    y = mean_values_df_with_calcs[quantity].values

    # keep only positive & finite for log10
    mask = (x > 0) & (y > 0) & np.isfinite(x) & np.isfinite(y)
    x = x[mask]
    y = y[mask]

    if len(x) < 2:
        continue

    # ---- linear fit in log10 space (power-law) ----
    logx = np.log10(x)
    logy = np.log10(y)

    slope, intercept, _, _, _ = linregress(logx, logy)
    # y = 10^intercept * x^slope
    x_fit = np.logspace(np.log10(x.min()), np.log10(x.max()), 200)
    y_fit = 10**intercept * x_fit**slope

    # ---- plot: original values, log–log axes ----
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=x, y=y, s=50, alpha=0.7,color='darkblue', label=quantity)
    plt.plot(x_fit, y_fit, color='darkred', linestyle='--',
             label=f'Power-law Fit (slope = {slope:.2f})')

    plt.xscale('log')
    plt.yscale('log')

    plt.title(f'{quantity} vs Distance (log–log scale)')
    plt.xlabel('Distance [AU]')
    ylabel = f"{quantity} [{UNITS_CALC.get(quantity, '')}]" if quantity in UNITS_CALC else quantity
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True, which='both', alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

# ---- Units for these quantities ----
UNITS_ai = {
    'Distance_mean': 'AU',

    # Temperatures
    'T_mean': 'K',
    'T_R_mean': 'K', 'T_T_mean': 'K', 'T_N_mean': 'K',
    'T_parallel3': 'K', 'T_perp3': 'K',

    # Plasma / field
    'N_mean': r'cm$^{-3}$',
    '|B|_mean': 'nT',

    # Invariants (adjust if you used different definitions)
    'magnetic_moment': 'K/nT',                 # T_perp / B
    'second_adiabatic_invariant': r'K nT$^{2}$ cm$^{-6}$'  # T_parallel * B^2 / N^2
}


def plot_2_ys(x, y1, y2, label_1, label_2, title_base, xlabel, ylabel):
    """
    Make TWO plots:
    1) Linear axes + linear fits for y1, y2
    2) Log–log axes + power-law fits for y1, y2
    """

    x = np.asarray(x)
    y1 = np.asarray(y1)
    y2 = np.asarray(y2)

    # =========================================================
    # 1) LINEAR PLOT + LINEAR FITS (WITH SORTING)
    # =========================================================
    mask1_lin = np.isfinite(x) & np.isfinite(y1)
    mask2_lin = np.isfinite(x) & np.isfinite(y2)

    x1_lin, y1_lin = x[mask1_lin], y1[mask1_lin]
    x2_lin, y2_lin = x[mask2_lin], y2[mask2_lin]

    if len(x1_lin) < 2 or len(x2_lin) < 2:
        return

    # Linear regressions
    slope1, intercept1, _, _, _ = linregress(x1_lin, y1_lin)
    slope2, intercept2, _, _, _ = linregress(x2_lin, y2_lin)

    # Sort for plotting
    idx1 = np.argsort(x1_lin)
    idx2 = np.argsort(x2_lin)

    x1_sorted, y1_sorted = x1_lin[idx1], y1_lin[idx1]
    x2_sorted, y2_sorted = x2_lin[idx2], y2_lin[idx2]

    y1_fit = slope1 * x1_sorted + intercept1
    y2_fit = slope2 * x2_sorted + intercept2

    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=x1_sorted, y=y1_sorted, label=label_1, color='darkred')
    plt.plot(x1_sorted, y1_fit, 'r--',
             label=f'Linear fit {label_1} (slope = {slope1:.2f})')

    sns.scatterplot(x=x2_sorted, y=y2_sorted, label=label_2, color='darkblue')
    plt.plot(x2_sorted, y2_fit, 'b--',
             label=f'Linear fit {label_2} (slope = {slope2:.2f})')

    plt.title(title_base)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # =========================================================
    # 2) LOG–LOG PLOT + POWER-LAW FITS (WITH SORTING)
    # =========================================================
    mask1_log = (x > 0) & (y1 > 0) & np.isfinite(x) & np.isfinite(y1)
    mask2_log = (x > 0) & (y2 > 0) & np.isfinite(x) & np.isfinite(y2)

    x1_log, y1_log = x[mask1_log], y1[mask1_log]
    x2_log, y2_log = x[mask2_log], y2[mask2_log]

    if len(x1_log) < 2 or len(x2_log) < 2:
        return

    logx1, logy1 = np.log10(x1_log), np.log10(y1_log)
    logx2, logy2 = np.log10(x2_log), np.log10(y2_log)

    slope1_p, intercept1_p, _, _, _ = linregress(logx1, logy1)
    slope2_p, intercept2_p, _, _, _ = linregress(logx2, logy2)

    # Create sorted x-grid for smooth power-law lines
    x_min = min(x1_log.min(), x2_log.min())
    x_max = max(x1_log.max(), x2_log.max())
    x_fit_log = np.logspace(np.log10(x_min), np.log10(x_max), 300)

    y1_fit_log = 10**intercept1_p * x_fit_log**slope1_p
    y2_fit_log = 10**intercept2_p * x_fit_log**slope2_p

    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=x1_log, y=y1_log, label=label_1, color='darkred')
    sns.scatterplot(x=x2_log, y=y2_log, label=label_2, color='darkblue')

    plt.plot(x_fit_log, y1_fit_log, 'r--',
             label=f'Power-law fit {label_1} (slope = {slope1_p:.2f})')
    plt.plot(x_fit_log, y2_fit_log, 'b--',
             label=f'Power-law fit {label_2} (slope = {slope2_p:.2f})')

    plt.xscale('log')
    plt.yscale('log')

    plt.title(title_base + " (log–log scale)")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True, which='both', alpha=0.3)
    plt.tight_layout()
    plt.show()


# ---------------- CALLS ----------------

x = mean_values_df_with_calcs['Distance_mean']

# 1) T_perp3 vs magnetic_moment
y1 = mean_values_df_with_calcs['T_perp3']
y2 = mean_values_df_with_calcs['magnetic_moment']
plot_2_ys(
    x, y1, y2,
    'T_perp3 [K]', 'magnetic_moment [K/nT]',
    "T_perp3 and magnetic_moment vs Distance",
    'Distance [AU]', 'Values'
)

# 2) T_mean vs magnetic_moment
y1 = mean_values_df_with_calcs['T_mean']
y2 = mean_values_df_with_calcs['magnetic_moment']
plot_2_ys(x, y1, y2,'T_mean [K]', 'magnetic_moment [K/nT]',"T_mean and magnetic_moment vs Distance",'Distance [AU]', 'Values')

# 3) T_parallel3 vs second_adiabatic_invariant
y1 = mean_values_df_with_calcs['T_parallel3']
y2 = mean_values_df_with_calcs['second_adiabatic_invariant']
plot_2_ys(x, y1, y2,'T_parallel3 [K]', 'second_adiabatic_invariant [K nT$^{2}$ cm$^{-6}$]',"T_parallel3 and second_adiabatic_invariant vs Distance", 'Distance [AU]', 'Values')

# 4) T_mean vs second_adiabatic_invariant
y1 = mean_values_df_with_calcs['T_mean']
y2 = mean_values_df_with_calcs['second_adiabatic_invariant']
plot_2_ys(
    x, y1, y2,
    'T_mean [K]', 'second_adiabatic_invariant [K nT$^{2}$ cm$^{-6}$]',
    "T_mean and second_adiabatic_invariant vs Distance",
    'Distance [AU]', 'Values'
)


# plots with Peroni et al equations

In [ ]:
peroni_et_al_df = mean_values_df_with_calcs.copy()

In [ ]:
def formula_peroni(m, m_uncertainty, R, R0, power, power_uncertainty):

    value = m * ((R / R0) ** (power))
    all_possible_uncertainties_taken_into_account = [
        (m + m_uncertainty) * ((R / R0) ** (power + power_uncertainty)),
        (m + m_uncertainty) * ((R / R0) ** (power - power_uncertainty)),
        (m - m_uncertainty) * ((R / R0) ** (power + power_uncertainty)),
        (m - m_uncertainty) * ((R / R0) ** (power - power_uncertainty)),
    ]
    # Calculate the formula
    return value, min(all_possible_uncertainties_taken_into_account), max(all_possible_uncertainties_taken_into_account)


peroni_et_al_df[['eq_n', 'eq_n_lower', 'eq_n_upper']] = peroni_et_al_df.apply(lambda row:
    formula_peroni(2.4, 0.1, row['Distance_mean'], 1, -1.96, 0.07), axis=1, result_type="expand")

peroni_et_al_df[['eq_B', 'eq_B_lower', 'eq_B_upper']] = peroni_et_al_df.apply(lambda row:
    formula_peroni(5.7, 0.2, row['Distance_mean'], 1, -1.59, 0.06), axis=1, result_type="expand")

peroni_et_al_df[['eq_BR', 'eq_BR_lower', 'eq_BR_upper']] = peroni_et_al_df.apply(lambda row:
    formula_peroni(3.5, 0.2, row['Distance_mean'], 1, -1.81, 0.08), axis=1, result_type="expand")

peroni_et_al_df[['eq_BT', 'eq_BT_lower', 'eq_BT_upper']] = peroni_et_al_df.apply(lambda row:
     formula_peroni(2.9, 0.2, row['Distance_mean'], 1, -1.21, 0.09), axis=1, result_type="expand")

peroni_et_al_df[['eq_T', 'eq_T_lower', 'eq_T_upper']] = peroni_et_al_df.apply(lambda row:
     formula_peroni(1.9*10**5, 0.1*10**5, row['Distance_mean'], 1, -0.9, 0.1), axis=1, result_type="expand")

peroni_et_al_df[['eq_T_parallel', 'eq_T_parallel_lower', 'eq_T_parallel_upper']] = peroni_et_al_df.apply(lambda row:
     formula_peroni(1.2*10**5, 0.1*10**5, row['Distance_mean'], 1, -0.5, 0.1), axis=1, result_type="expand")

peroni_et_al_df[['eq_T_perp', 'eq_T_perp_lower', 'eq_T_perp_upper']] = peroni_et_al_df.apply(lambda row:
     formula_peroni(2.3*10**5, 0.1*10**5, row['Distance_mean'], 1, -1.0, 0.1), axis=1, result_type="expand")

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

# Define the quantities to plot:
# [data_col, perrone_mean_col, perrone_lower_col, perrone_upper_col]
quantities = [
    ['T_mean',      'eq_T',          'eq_T_lower',          'eq_T_upper'],
    ['T_parallel3', 'eq_T_parallel', 'eq_T_parallel_lower', 'eq_T_parallel_upper'],
    ['T_perp3',     'eq_T_perp',     'eq_T_perp_lower',     'eq_T_perp_upper'],
    ['|B|_mean',    'eq_B',          'eq_B_lower',          'eq_B_upper'],
    ['BR_mean',     'eq_BR',         'eq_BR_lower',         'eq_BR_upper'],
    ['BT_mean',     'eq_BT',         'eq_BT_lower',         'eq_BT_upper'],
    ['N_mean',      'eq_n',          'eq_n_lower',          'eq_n_upper'],
]


for quantity in quantities:
    data_col, eq_col, eq_lower_col, eq_upper_col = quantity
    print(data_col)

    # ------------------- Get data in physical space -------------------
    x = peroni_et_al_df['Distance_mean'].values
    y = peroni_et_al_df[data_col].values
    y_eq = peroni_et_al_df[eq_col].values
    y_lower = peroni_et_al_df[eq_lower_col].values
    y_upper = peroni_et_al_df[eq_upper_col].values

    # ================================================================
    # 1) LINEAR PLOT + LINEAR FIT (data) + Perrone mean & bounds
    # ================================================================
    mask_data_lin = np.isfinite(x) & np.isfinite(y)
    x_data_lin = x[mask_data_lin]
    y_data_lin = y[mask_data_lin]

    if len(x_data_lin) < 2:
        continue

    slope_lin, intercept_lin, _, _, _ = linregress(x_data_lin, y_data_lin)

    # Sort for plotting
    idx_data_lin = np.argsort(x_data_lin)
    x_data_sorted = x_data_lin[idx_data_lin]
    y_data_sorted = y_data_lin[idx_data_lin]
    y_fit_lin = slope_lin * x_data_sorted + intercept_lin

    # Perrone curves: sort them too (they may have NaNs at edges)
    mask_eq_lin = np.isfinite(x) & np.isfinite(y_eq) & np.isfinite(y_lower) & np.isfinite(y_upper)
    x_eq_lin = x[mask_eq_lin]
    y_eq_lin = y_eq[mask_eq_lin]
    y_lower_lin = y_lower[mask_eq_lin]
    y_upper_lin = y_upper[mask_eq_lin]

    idx_eq_lin = np.argsort(x_eq_lin)
    x_eq_sorted = x_eq_lin[idx_eq_lin]
    y_eq_sorted = y_eq_lin[idx_eq_lin]
    y_lower_sorted = y_lower_lin[idx_eq_lin]
    y_upper_sorted = y_upper_lin[idx_eq_lin]

    # --------- Linear figure ---------
    plt.figure(figsize=(8, 6))
    # Data
    sns.scatterplot(x=x_data_sorted, y=y_data_sorted,
                    s=50, alpha=0.7, label=data_col, color='#4E79A7')
    plt.plot(x_data_sorted, y_fit_lin,
             color='#4E79A7', alpha=0.7, linestyle='--',
             linewidth=1.5,
             label=f'Linear Fit (slope = {slope_lin:.2f})')

    # Perrone mean and bounds (physical space)
    plt.plot(x_eq_sorted, y_eq_sorted,
             color='blue', linestyle='-',
             label=f'Perrone Mean ({eq_col})')
    plt.fill_between(x_eq_sorted, y_lower_sorted, y_upper_sorted,
                     color='blue', alpha=0.2, label='Perrone Bounds')

    plt.title(f'{data_col} vs Distance with Perrone Bounds')
    plt.xlabel('Distance [AU]')
    plt.ylabel(data_col)
    plt.legend()
    plt.grid(True, which='both', alpha=0.3)
    plt.tight_layout()
    plt.show()

    # ================================================================
    # 2) LOG–LOG PLOT + POWER-LAW FIT (data) + Perrone mean & bounds
    # ================================================================
    # Require positive & finite for logs
    mask_data_log = (x > 0) & (y > 0) & np.isfinite(x) & np.isfinite(y)
    x_data_log = x[mask_data_log]
    y_data_log = y[mask_data_log]

    if len(x_data_log) < 2:
        continue

    logx = np.log10(x_data_log)
    logy = np.log10(y_data_log)

    slope_pow, intercept_pow, _, _, _ = linregress(logx, logy)
    # Power-law: y = 10^intercept_pow * x^slope_pow
    x_fit_log = np.logspace(np.log10(x_data_log.min()),
                            np.log10(x_data_log.max()), 300)
    y_fit_log = 10**intercept_pow * x_fit_log**slope_pow

    # Perrone curves for log–log: only positive & finite
    mask_eq_log = (x > 0) & (y_eq > 0) & (y_lower > 0) & (y_upper > 0) & \
                  np.isfinite(x) & np.isfinite(y_eq) & np.isfinite(y_lower) & np.isfinite(y_upper)
    x_eq_log = x[mask_eq_log]
    y_eq_log = y_eq[mask_eq_log]
    y_lower_log = y_lower[mask_eq_log]
    y_upper_log = y_upper[mask_eq_log]

    idx_eq_log = np.argsort(x_eq_log)
    x_eq_log_sorted = x_eq_log[idx_eq_log]
    y_eq_log_sorted = y_eq_log[idx_eq_log]
    y_lower_log_sorted = y_lower_log[idx_eq_log]
    y_upper_log_sorted = y_upper_log[idx_eq_log]

    # --------- Log–log figure ---------
    plt.figure(figsize=(8, 6))
    # Data points (physical values)
    sns.scatterplot(x=x_data_log, y=y_data_log,
                    s=50, alpha=0.7, label=data_col, color='#4E79A7')

    # Power-law fit
    plt.plot(x_fit_log, y_fit_log,
             color='#4E79A7', alpha=0.7, linestyle='--',
             linewidth=1.5,
             label=f'Power-law Fit (slope = {slope_pow:.2f})')

    # Perrone mean and bounds (physical, on log axes)
    if len(x_eq_log_sorted) > 0:
        plt.plot(x_eq_log_sorted, y_eq_log_sorted,
                 color='blue', linestyle='-',
                 label=f'Perrone Mean ({eq_col})')
        plt.fill_between(x_eq_log_sorted,
                         y_lower_log_sorted, y_upper_log_sorted,
                         color='blue', alpha=0.2, label='Perrone Bounds')

    plt.xscale('log')
    plt.yscale('log')

    plt.title(f'{data_col} vs Distance (log–log) with Perrone Bounds')
    plt.xlabel('Distance [AU]')
    plt.ylabel(data_col)
    plt.legend()
    plt.grid(True, which='both', alpha=0.3)
    plt.tight_layout()
    plt.show()


# scatter plots for sphere relation 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

# ---- Units for these quantities ----
UNITS = {'BR': 'nT', 'BT': 'nT', 'BN': 'nT'}


# ----------------------------------------------------
# Main plotting function
# ----------------------------------------------------

x = MAG_df['BN'].values
y = MAG_df['BR'].values

# =========================================================
# 1) LINEAR PLOT
# =========================================================
mask_lin = np.isfinite(x) & np.isfinite(y)


x_lin = x[mask_lin]
y_lin = y[mask_lin]

plt.figure()
sns.scatterplot(x=x_lin, y=y_lin, label="Date_range_timestamps",alpha=0.5)


plt.title('$B_R$ vs $B_N$')
plt.xlabel(f"$B_N$ [{UNITS['BN']}]")
plt.ylabel(f"$B_R$ [{UNITS['BR']}]")
plt.grid(True, which='both', alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# # =========================================================
# # 2) LOG–LOG PLOT
# # =========================================================
# 
# mask_log = (x > 0) & (y > 0) & np.isfinite(x) & np.isfinite(y)
# # if mask_log.sum() < 2:
# #     continue
# 
# x_log = x[mask_log]
# y_log = y[mask_log]
# 
# logx = np.log10(x_log)
# logy = np.log10(y_log)
# 
# 
# # x_fit_log = np.logspace(np.log10(x_log.min()),
# #                         np.log10(x_log.max()), 200)
# # y_fit_log = 10**intercept_pow * x_fit_log**slope_pow
# 
# plt.figure()
# sns.scatterplot(x=x_log, y=y_log, label=col)
# 
# 
# plt.xscale('log')
# plt.yscale('log')
# 
# # FORCE PRETTY Y-AXIS RANGE (≥ 3 ROUND LOG NUMBERS)
# ymin = y_log.min()
# ymax = y_log.max()
# 
# log_ymin = np.floor(np.log10(ymin))
# log_ymax = np.ceil(np.log10(ymax))
# 
# # Expand if less than 2 decades
# if (log_ymax - log_ymin) < 1:
#     log_ymid = 0.5 * (log_ymin + log_ymax)
#     log_ymin = np.floor(log_ymid - 1)
#     log_ymax = np.ceil(log_ymid + 1)
# 
# plt.ylim(10**log_ymin, 10**log_ymax)
# 
# plt.title('BR vs BN')
# plt.xlabel(f"BN [{UNITS['BN']}]")
# plt.ylabel(f"BN [{UNITS['BR']}]")
# plt.grid(True, which='both', alpha=0.3)
# plt.legend()
# plt.tight_layout()
# plt.show()

# ----------------------------------------------------
# Run it
 # ----------------------------------------------------
MAG_df

In [ ]:
MAG_df

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

UNITS = {'BR': 'nT', 'BT': 'nT', 'BN': 'nT'}

stream_name = "Date_Range3"   # change this to browse streams

df_stream = MAG_df[MAG_df["name"] == stream_name].copy()

x = df_stream["BN"].to_numpy()
y = df_stream["BR"].to_numpy()

mask_lin = np.isfinite(x) & np.isfinite(y)


x_lin = x[mask_lin]
y_lin = y[mask_lin]
plt.figure()
plt.scatter(x, y, s=5, alpha=0.5, label="Date_Range1_timestamps")


plt.title(f"{stream_name}: $B_R$ vs $B_N$")
plt.xlabel(f"$B_N$ [{UNITS['BN']}]")
plt.ylabel(f"$B_R$ [{UNITS['BR']}]")
plt.grid(True, which='both', alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
MAG_df

In [ ]:
MAG_df['solar radii']=MAG_df['Distance']*215 # 1au=215 solar radii
print(MAG_df[['Distance','solar radii']])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ----------------------------------------------------
# Units (only for labels)
# ----------------------------------------------------
UNITS = {'BR': 'nT', 'BT': 'nT', 'Distance': 'AU'}

# ----------------------------------------------------
# Distance bins in AU
# ----------------------------------------------------
au_bins = [0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1.00]

# ----------------------------------------------------
# Load raw data
# ----------------------------------------------------
BT = MAG_df['BT'].values      # x-axis (nT)
BR = MAG_df['BR'].values      # y-axis (nT)
R  = MAG_df['Distance'].values  # AU

# ----------------------------------------------------
# Remove NaN / inf values
# ----------------------------------------------------
mask = np.isfinite(BT) & np.isfinite(BR) & np.isfinite(R)
BT = BT[mask]
BR = BR[mask]
R  = R[mask]

# ====================================================
# 1) Distance-binned raw hodogram
# ====================================================
plt.figure(figsize=(7, 6))

for i in range(len(au_bins) - 1):
    lo, hi = au_bins[i], au_bins[i + 1] #bin ranges

    in_bin = (R >= lo) & (R < hi) #Select values of R that are greater than or equal to lo AND less than hi and put them in a list of True / False values.

     #skip empty bins
    if in_bin.sum() == 0:
        continue

#plots only the points that belong to this distance bin.
    plt.scatter(
        BT[in_bin], #Give me only the BT values where in_bin is True.
        BR[in_bin], #Give me only the BR values where in_bin is True.
        s=1, #dot size
        alpha=0.4,#dot transparency #sparse regions look light,dense regions look darker
        label=f"{lo:.2f}–{hi:.2f} AU"
    )
 

plt.title(r"$B_R$ vs $B_T$ (R binned)")
plt.xlabel(f"$B_T$ [{UNITS['BT']}]")
plt.ylabel(f"$B_R$ [{UNITS['BR']}]")
plt.grid(alpha=0.3)
plt.legend(title=f"R [{UNITS['Distance']}]", markerscale=4)
plt.tight_layout()
plt.show()

# # ====================================================
# # 2) Optional: time direction arrow
# # ====================================================
# do_time_arrow = True
# 
# if do_time_arrow and len(BT) > 1000:
# 
#     n = int(0.05 * len(BT))   # first & last 5%
# 
#     x0, y0 = np.mean(BT[:n]), np.mean(BR[:n])
#     x1, y1 = np.mean(BT[-n:]), np.mean(BR[-n:])
# 
#     plt.figure(figsize=(7, 6))
#     plt.scatter(BT, BR, s=3, alpha=0.05)
# 
#     plt.arrow(
#         x0, y0,
#         x1 - x0,
#         y1 - y0,
#         length_includes_head=True,
#         head_width=0.2,
#         head_length=0.2
#     )
# 
#     plt.text(x0, y0, "time", fontsize=12)
# 
#     plt.title(r"$B_R$ vs $B_T$ (time direction)")
#     plt.xlabel(f"$B_T$ [{UNITS['BT']}]")
#     plt.ylabel(f"$B_R$ [{UNITS['BR']}]")
#     plt.grid(alpha=0.3)
#     plt.tight_layout()
#     plt.show()


# plots to check 5 relations 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

# ----------------- Units -----------------
UNITS_deltas = {
    'Distance': 'AU',
    'delta_B_parallel_0': 'nT',
    'delta_B_perp_0': 'nT'
}

# ----------------- Main plotting function -----------------
def redo_all_delta_means_plots(df):

    # ONLY these two columns, and they must be on ONE figure
    columns = ['delta_B_parallel_0', 'delta_B_perp_0']

    x = df['Distance'].values
    y1 = df[columns[0]].values
    y2 = df[columns[1]].values

    # ---------------- 1) Linear plot + linear fits (ONE FIGURE) ----------------
    mask_lin = np.isfinite(x) & np.isfinite(y1) & np.isfinite(y2)
    if mask_lin.sum() < 2:
        return

    x_lin = x[mask_lin]
    y1_lin = y1[mask_lin]
    y2_lin = y2[mask_lin]

    slope_1, intercept_1, _, _, _ = linregress(x_lin, y1_lin)
    slope_2, intercept_2, _, _, _ = linregress(x_lin, y2_lin)

    x_fit_lin = np.linspace(x_lin.min(), x_lin.max(), 200)
    y1_fit = slope_1 * x_fit_lin + intercept_1
    y2_fit = slope_2 * x_fit_lin + intercept_2

    plt.figure()

    sns.scatterplot(x=x_lin, y=y1_lin, color='darkred', label='delta_B_parallel_0')
    sns.scatterplot(x=x_lin, y=y2_lin, color='darkblue', label='delta_B_perp_0')

    plt.plot(x_fit_lin, y1_fit, 'r--',
             label=f'Linear Fit parallel (slope = {slope_1:.2f})')
    plt.plot(x_fit_lin, y2_fit, 'b--',
             label=f'Linear Fit perp (slope = {slope_2:.2f})')

    plt.title('delta_B_parallel_0 and delta_B_perp_0 vs Distance')
    plt.xlabel(f"Distance [{UNITS_deltas['Distance']}]")
    plt.ylabel("delta_B [nT]")

    plt.grid(True, which='both', alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # ---------------- 2) Log–log plot + power-law fits (ONE FIGURE) ----------------
    mask_log = (x > 0) & (y1 > 0) & (y2 > 0) & np.isfinite(x) & np.isfinite(y1) & np.isfinite(y2)
    if mask_log.sum() < 2:
        return

    x_log = x[mask_log]
    y1_log = y1[mask_log]
    y2_log = y2[mask_log]

    logx = np.log10(x_log)
    logy1 = np.log10(y1_log)
    logy2 = np.log10(y2_log)

    slope_1_pow, intercept_1_pow, _, _, _ = linregress(logx, logy1)
    slope_2_pow, intercept_2_pow, _, _, _ = linregress(logx, logy2)

    x_fit_log = np.logspace(np.log10(x_log.min()), np.log10(x_log.max()), 200)
    y1_fit_log = 10**intercept_1_pow * x_fit_log**slope_1_pow
    y2_fit_log = 10**intercept_2_pow * x_fit_log**slope_2_pow

    plt.figure()

    sns.scatterplot(x=x_log, y=y1_log, color='darkred', label='delta_B_parallel_0')
    sns.scatterplot(x=x_log, y=y2_log, color='darkblue', label='delta_B_perp_0')

    plt.plot(x_fit_log, y1_fit_log, 'r--',
             label=f'Power-law Fit parallel (slope = {slope_1_pow:.2f})')
    plt.plot(x_fit_log, y2_fit_log, 'b--',
             label=f'Power-law Fit perp (slope = {slope_2_pow:.2f})')

    plt.xscale('log')
    plt.yscale('log')

    # ---------   y-limits expansion ---------
    ymin = min(y1_log.min(), y2_log.min())
    ymax = max(y1_log.max(), y2_log.max())

    log_ymin = np.floor(np.log10(ymin))
    log_ymax = np.ceil(np.log10(ymax))

    # Expand if less than 2 digits (i.e. less than 2 decades)
    if (log_ymax - log_ymin) < 1:
        log_ymid = 0.5 * (log_ymin + log_ymax)
        log_ymin = np.floor(log_ymid - 1)
        log_ymax = np.ceil(log_ymid + 1)

    plt.ylim(10**log_ymin, 10**log_ymax)
    # --------------------------------------------------------------------

    plt.title('delta_B_parallel_0 and delta_B_perp_0 vs Distance (log–log scale)')
    plt.xlabel(f"Distance [{UNITS_deltas['Distance']}]")
    plt.ylabel("delta_B [nT]")

    plt.grid(True, which='both', alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

    print("slope_parallel =",slope_1_pow,"intercept_parallel =", intercept_1_pow)
    print("slope_perp =",slope_2_pow,"intercept_perp =", intercept_2_pow)


# Example call:
redo_all_delta_means_plots(all_delta_means_withmeans)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

# ----------------- Units -----------------
UNITS_deltas = {
    'Distance': 'AU'
}

# ----------------- Main plotting function -----------------
def redo_all_delta_means_plots(df):

    # ONLY these two columns, and they must be on ONE figure
    columns = ['delta_B_parallel_0/B_0', 'delta_B_perp_0/B_0']

    x = df['Distance'].values
    y1 = df[columns[0]].values
    y2 = df[columns[1]].values

    # ---------------- 1) Linear plot + linear fits (ONE FIGURE) ----------------
    mask_lin = np.isfinite(x) & np.isfinite(y1) & np.isfinite(y2)
    if mask_lin.sum() < 2:
        return

    x_lin = x[mask_lin]
    y1_lin = y1[mask_lin]
    y2_lin = y2[mask_lin]

    slope_1, intercept_1, _, _, _ = linregress(x_lin, y1_lin)
    slope_2, intercept_2, _, _, _ = linregress(x_lin, y2_lin)

    x_fit_lin = np.linspace(x_lin.min(), x_lin.max(), 200)
    y1_fit = slope_1 * x_fit_lin + intercept_1
    y2_fit = slope_2 * x_fit_lin + intercept_2

    plt.figure()

    sns.scatterplot(x=x_lin, y=y1_lin, color='darkred', label='delta_B_parallel_0/B_0')
    sns.scatterplot(x=x_lin, y=y2_lin, color='darkblue', label='delta_B_perp_0/B_0')

    plt.plot(x_fit_lin, y1_fit, 'r--',
             label=f'Linear Fit parallel (slope = {slope_1:.2f})')
    plt.plot(x_fit_lin, y2_fit, 'b--',
             label=f'Linear Fit perp (slope = {slope_2:.2f})')

    plt.title('delta_B_parallel_0/B_0 and delta_B_perp_0/B_0 vs Distance')
    plt.xlabel(f"Distance [{UNITS_deltas['Distance']}]")
    plt.ylabel("delta_B/B_0")

    plt.grid(True, which='both', alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # ---------------- 2) Log–log plot + power-law fits (ONE FIGURE) ----------------
    mask_log = (x > 0) & (y1 > 0) & (y2 > 0) & np.isfinite(x) & np.isfinite(y1) & np.isfinite(y2)
    if mask_log.sum() < 2:
        return

    x_log = x[mask_log]
    y1_log = y1[mask_log]
    y2_log = y2[mask_log]

    logx = np.log10(x_log)
    logy1 = np.log10(y1_log)
    logy2 = np.log10(y2_log)

    slope_1_pow, intercept_1_pow, _, _, _ = linregress(logx, logy1)
    slope_2_pow, intercept_2_pow, _, _, _ = linregress(logx, logy2)

    x_fit_log = np.logspace(np.log10(x_log.min()), np.log10(x_log.max()), 200)
    y1_fit_log = 10**intercept_1_pow * x_fit_log**slope_1_pow
    y2_fit_log = 10**intercept_2_pow * x_fit_log**slope_2_pow

    plt.figure()

    sns.scatterplot(x=x_log, y=y1_log, color='darkred', label='delta_B_parallel_0/B_0')
    sns.scatterplot(x=x_log, y=y2_log, color='darkblue', label='delta_B_perp_0/B_0')

    plt.plot(x_fit_log, y1_fit_log, 'r--',
             label=f'Power-law Fit parallel (slope = {slope_1_pow:.2f})')
    plt.plot(x_fit_log, y2_fit_log, 'b--',
             label=f'Power-law Fit perp (slope = {slope_2_pow:.2f})')

    plt.xscale('log')
    plt.yscale('log')

    # --------- ADDED y-axis expansion block ---------
    ymin = min(y1_log.min(), y2_log.min())
    ymax = max(y1_log.max(), y2_log.max())

    log_ymin = np.floor(np.log10(ymin))
    log_ymax = np.ceil(np.log10(ymax))

    if (log_ymax - log_ymin) < 1:
        log_ymid = 0.5 * (log_ymin + log_ymax)
        log_ymin = np.floor(log_ymid - 1)
        log_ymax = np.ceil(log_ymid + 1)

    plt.ylim(10**log_ymin, 10**log_ymax)
    # -----------------------------------------------

    plt.title('delta_B_parallel_0/B_0 and delta_B_perp_0/B_0 vs Distance (log–log scale)')
    plt.xlabel(f"Distance [{UNITS_deltas['Distance']}]")
    plt.ylabel("delta_B/B_0")

    plt.grid(True, which='both', alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()


# Example call:
redo_all_delta_means_plots(all_delta_means_withmeans)


In [ ]:
all_delta_means_withmeans